In [1]:
# @title
# Instalar dependencias (ejecutar solo una vez por sesión)
!pip install pandas pydaymet datetime matplotlib ipywidgets openpyxl > /dev/null

import pandas as pd
from pydaymet import get_bycoords
from datetime import date
import matplotlib.pyplot as plt
from ipywidgets import FloatText, IntText, Text, IntSlider, Button, HBox, VBox, Output, Checkbox
from IPython.display import display
from google.colab import files

def coordenadas_validas(lat, lon):
    # Validar rango de coordenadas para Daymet
    return 14.0 <= lat <= 52.0 and -130.0 <= lon <= -52.5

def get_data_and_yearrep(lat, lon, CN_base):
    # Descargar datos diarios de Daymet y calcular año representativo
    start = date(1980, 1, 1)
    end = date(date.today().year - 1, 12, 31)
    dates = (start.strftime("%Y-%m-%d"), end.strftime("%Y-%m-%d"))
    df = get_bycoords((lon, lat), dates, variables=['prcp'], time_scale='daily').reset_index()
    df = df.rename(columns={'time':'Date', 'prcp (mm/day)':'PCP'})[['Date', 'PCP']]
    df['PCP'] = df['PCP'].interpolate(method='linear', limit=1)
    df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d', errors='coerce')
    df = df.dropna(subset=['Date'])
    df['Year'] = df['Date'].dt.year
    df['Month'] = df['Date'].dt.month

    # Calcular escurrimiento base usando el método SCS-CN
    S_base = 25400 / CN_base - 254
    def runoff(P, S):
        return ((P - 0.2*S)**2)/(P + 0.8*S) if P > 0.2*S else 0
    df['Q_base'] = df['PCP'].apply(lambda P: runoff(P, S_base))

    # Encontrar año representativo según mediana anual de escurrimiento base
    annual_base = df.groupby('Year')['Q_base'].sum().reset_index(name='annual_Q_base')
    median_base = annual_base['annual_Q_base'].median()
    annual_base['diff_mediana'] = (annual_base['annual_Q_base'] - median_base).abs()
    min_diff = annual_base['diff_mediana'].min()
    candidatos = annual_base[annual_base['diff_mediana'] == min_diff]
    year_rep = candidatos['Year'].max()
    return int(year_rep), int(df['Year'].min()), int(df['Year'].max()), df

def run_dashboard(nombre, lat, lon, CN_base, CN_obra, year, mes_ini, mes_fin, variables, output):
    # Limpiar salida previa
    output.clear_output()
    with output:
        # Validar coordenadas antes de consultar Daymet
        if not coordenadas_validas(lat, lon):
            print(f"\nCoordenadas inválidas para Daymet.\nLatitud válida: 14.0 a 52.0\nLongitud válida: -130.0 a -52.5")
            print(f"Latitud actual: {lat:.4f}, Longitud actual: {lon:.4f}")
            return

        # Obtener datos y año representativo
        year_rep, min_year, max_year, df = get_data_and_yearrep(lat, lon, CN_base)
        if year < min_year or year > max_year:
            print(f"Año {year} no disponible. Elegir entre {min_year} y {max_year}.")
            return

        # Calcular escurrimiento base y con obras
        S_base = 25400 / CN_base - 254
        S_obra = 25400 / CN_obra - 254
        def runoff(P, S):
            return ((P - 0.2*S)**2)/(P + 0.8*S) if P > 0.2*S else 0
        df['Q_base'] = df['PCP'].apply(lambda P: runoff(P, S_base))
        df['Q_obra'] = df['PCP'].apply(lambda P: runoff(P, S_obra))
        df['Evento_base'] = df['PCP'] > 0.2 * S_base
        df['Evento_obra'] = df['PCP'] > 0.2 * S_obra

        # Filtrar año completo para métricas y barras
        df_anno = df[df['Year'] == year]

        # Calcular métricas anuales
        q_base_year = df_anno['Q_base'].sum()
        q_obra_year = df_anno['Q_obra'].sum()
        diferencia_mm = q_base_year - q_obra_year
        diferencia_m3ha = diferencia_mm * 10
        eventos_base_year = df_anno['Evento_base'].sum()
        eventos_obra_year = df_anno['Evento_obra'].sum()
        pp_anual = df_anno['PCP'].sum()
        coef_q_base = q_base_year / pp_anual if pp_anual > 0 else 0
        coef_q_obra = q_obra_year / pp_anual if pp_anual > 0 else 0

        # Mostrar resumen de métricas anuales
        print(f"\nSitio: {nombre} ({lat:.4f}, {lon:.4f})")
        print(f"Año representativo (completo): {year_rep}")
        print(f"Año seleccionado (completo): {int(year)}")
        print(f"Escurrimiento base (año completo): {q_base_year:.2f} mm")
        print(f"Escurrimiento con obras (año completo): {q_obra_year:.2f} mm")
        print(f"Beneficio volumétrico (año completo): {diferencia_mm:.2f} mm   =   {diferencia_m3ha:.2f} m³/ha")
        print(f"Eventos con P > 0.2S base (año completo): {int(eventos_base_year)}")
        print(f"Eventos con P > 0.2S con obras (año completo): {int(eventos_obra_year)}")
        print(f"Coef. escurrimiento base: {coef_q_base:.2%}")
        print(f"Coef. escurrimiento con obras: {coef_q_obra:.2%}")

        # Filtrar rango de meses para hidrograma mostrado
        df_rep = df_anno[(df_anno['Month'] >= mes_ini) & (df_anno['Month'] <= mes_fin)]

        # Graficar hidrograma del rango de meses seleccionado
        plt.figure(figsize=(15,6))
        if 'Precipitación' in variables:
            plt.plot(df_rep['Date'], df_rep['PCP'], color='#0055B8', label='Precipitación (mm)', linewidth=1, alpha=0.6)
        if 'Escurrimiento base' in variables:
            plt.plot(df_rep['Date'], df_rep['Q_base'], color='#FF6469', label='Escurrimiento Base (mm)', linewidth=1)
        if 'Escurrimiento con obras' in variables:
            plt.plot(df_rep['Date'], df_rep['Q_obra'], color='#32CC68', label='Escurrimiento con obras (mm)', linewidth=1)
        plt.title(f'Hidrograma año representativo: {int(year)}, meses {mes_ini}-{mes_fin} - {nombre}')
        plt.xlabel('Fecha')
        plt.ylabel('mm')
        plt.legend()
        plt.grid(alpha=0.3)
        plt.tight_layout()
        plt.show()

        # Graficar barras comparativas anuales
        plt.figure(figsize=(10,5))
        plt.bar(['Base','Con obras','Beneficio'], [q_base_year, q_obra_year, diferencia_mm], color=['#FF6469','#80CE9F','#0055B8'], alpha=0.8)
        plt.ylabel('Escurrimiento anual (mm)')
        plt.title(f'Comparación escurrimiento y beneficio volumétrico\nAño representativo: {int(year)} - {nombre}')
        plt.grid(axis='y', alpha=0.2)
        plt.tight_layout()
        plt.show()

        # Preparar datos exportables: siempre exportar Fecha, Precipitación, Escurrimiento base y con obras
        global last_df_export
        export_cols = ['Date', 'PCP', 'Q_base', 'Q_obra']
        export_names = ['Fecha', 'Precipitación (mm)', 'Escurrimiento base (mm)', 'Escurrimiento con obras (mm)']
        last_df_export = df_anno[export_cols].sort_values('Date').copy()
        last_df_export.columns = export_names

# Definir widgets interactivos
nombre_default = "Toroto"
lat_default = 19.418922222134
lon_default = -99.16033686822948
CN_base_default = 90
CN_obra_default = 80

# Obtener año representativo y rango de años (con coordenadas por defecto válidas)
year_rep, min_year, max_year, df_init = get_data_and_yearrep(lat_default, lon_default, CN_base_default)
nombre = Text(value=nombre_default, description="Nombre:")
lat = FloatText(value=lat_default, description="Latitud:")
lon = FloatText(value=lon_default, description="Longitud:")
CN_base = IntText(value=CN_base_default, description="CN base:")
CN_obra = IntText(value=CN_obra_default, description="CN obras:")
year = IntText(value=year_rep, description="Año")
mes_ini = IntSlider(value=1, min=1, max=12, description="Mes inicial:", continuous_update=False)
mes_fin = IntSlider(value=12, min=1, max=12, description="Mes final:", continuous_update=False)
prec_widget = Checkbox(value=True, description='Precipitación')
base_widget = Checkbox(value=True, description='Escurrimiento base')
obra_widget = Checkbox(value=True, description='Escurrimiento con obras')
btn_rep = Button(description="Año representativo", button_style='info')
btn_export = Button(description="Descargar (.xlsx)", button_style='success')
output = Output()

last_df_export = None  # Variable global para exportación

def get_variables_selected():
    # Obtener variables seleccionadas para mostrar en la gráfica (nombres cortos)
    v = []
    if prec_widget.value:
        v.append('Precipitación')
    if base_widget.value:
        v.append('Escurrimiento base')
    if obra_widget.value:
        v.append('Escurrimiento con obras')
    return v

def actualizar_dashboard(change=None):
    # Actualizar dashboard cuando cambia algún parámetro
    if mes_ini.value > mes_fin.value:
        mes_ini.value, mes_fin.value = mes_fin.value, mes_ini.value
    run_dashboard(
        nombre.value, lat.value, lon.value, CN_base.value, CN_obra.value,
        year.value, mes_ini.value, mes_fin.value,
        get_variables_selected(), output
    )

for w in [nombre, lat, lon, CN_base, CN_obra, year, mes_ini, mes_fin, prec_widget, base_widget, obra_widget]:
    w.observe(actualizar_dashboard, names='value')

def set_year_rep(b):
    # Asignar año representativo como año seleccionado
    year_rep, _, _, _ = get_data_and_yearrep(lat.value, lon.value, CN_base.value)
    year.value = year_rep

btn_rep.on_click(set_year_rep)

def exportar_datos(b):
    # Exportar datos anuales a archivo Excel y descargar automáticamente con nombre automático
    global last_df_export
    fname = f"{nombre.value}_{year.value}.xlsx"
    if last_df_export is not None and len(last_df_export) > 0:
        last_df_export.to_excel(fname, index=False)
        files.download(fname)
    else:
        print("No hay datos para exportar. Cambiar parámetros y volver a intentar.")

btn_export.on_click(exportar_datos)

# Mostrar interfaz gráfica interactiva
display(VBox([
    HBox([nombre, lat, lon]),
    HBox([CN_base, CN_obra, year, btn_rep]),
    HBox([mes_ini, mes_fin]),
    HBox([prec_widget, base_widget, obra_widget]),
    btn_export,
    output
]))
actualizar_dashboard()

El sistema no puede encontrar la ruta especificada.


ModuleNotFoundError: No module named 'google.colab'

# Tries

In [8]:
# @title
# Instala dependencias (si no las tienes) - ejecuta sólo una vez por sesión
!pip install pandas pydaymet datetime matplotlib ipywidgets > /dev/null

import pandas as pd
from pydaymet import get_bycoords
from datetime import date
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatText, IntText, Text, IntSlider

def get_data_and_yearrep(lat, lon, CN_base):
    # Descargar datos y calcular año representativo
    start = date(1980, 1, 1)
    end = date(date.today().year - 1, 12, 31)
    dates = (start.strftime("%Y-%m-%d"), end.strftime("%Y-%m-%d"))
    df = get_bycoords((lon, lat), dates, variables=['prcp'], time_scale='daily').reset_index()
    df = df.rename(columns={'time':'Date', 'prcp (mm/day)':'PCP'})[['Date', 'PCP']]
    df['PCP'] = df['PCP'].interpolate(method='linear', limit=1)
    df['Date'] = pd.to_datetime(df['Date']).dt.strftime('%d/%m/%Y')
    df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')
    df['Year'] = df['Date'].dt.year

    S_base = 25400 / CN_base - 254
    def runoff(P, S):
        return ((P - 0.2*S)**2)/(P + 0.8*S) if P > 0.2*S else 0
    df['Q_base'] = df['PCP'].apply(lambda P: runoff(P, S_base))

    annual_base = df.groupby('Year')['Q_base'].sum().reset_index(name='annual_Q_base')
    median_base = annual_base['annual_Q_base'].median()
    annual_base['diff_mediana'] = (annual_base['annual_Q_base'] - median_base).abs()
    min_diff = annual_base['diff_mediana'].min()
    candidatos = annual_base[annual_base['diff_mediana'] == min_diff]
    year_rep = candidatos['Year'].max()
    # También retorna los años disponibles
    return int(year_rep), int(df['Year'].min()), int(df['Year'].max())

def run_dashboard(nombre, lat, lon, CN_base, CN_obra, year):
    # === 1. Fechas y datos ===
    start = date(1980, 1, 1)
    end = date(date.today().year - 1, 12, 31)
    dates = (start.strftime("%Y-%m-%d"), end.strftime("%Y-%m-%d"))
    df = get_bycoords((lon, lat), dates, variables=['prcp'], time_scale='daily').reset_index()
    df = df.rename(columns={'time':'Date', 'prcp (mm/day)':'PCP'})[['Date', 'PCP']]
    df['PCP'] = df['PCP'].interpolate(method='linear', limit=1)
    df['Date'] = pd.to_datetime(df['Date']).dt.strftime('%d/%m/%Y')
    df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')
    df['Year'] = df['Date'].dt.year

    # === 2. Cálculo de escurrimiento ===
    def runoff(P, S):
        return ((P - 0.2*S)**2)/(P + 0.8*S) if P > 0.2*S else 0

    S_base = 25400 / CN_base - 254
    S_obra = 25400 / CN_obra - 254

    df['Q_base'] = df['PCP'].apply(lambda P: runoff(P, S_base))
    df['Q_obra'] = df['PCP'].apply(lambda P: runoff(P, S_obra))

    annual_base = df.groupby('Year')['Q_base'].sum().reset_index(name='annual_Q_base')
    annual_obra = df.groupby('Year')['Q_obra'].sum().reset_index(name='annual_Q_obra')

    median_base = annual_base['annual_Q_base'].median()

    # === 3. Si el año no existe, avisa ===
    if year not in annual_base['Year'].values:
        print(f"Año {year} no disponible. Elige entre {annual_base['Year'].min()} y {annual_base['Year'].max()}.")
        return

    q_base_rep = annual_base[annual_base['Year'] == year]['annual_Q_base'].values[0]
    q_obra_rep = annual_obra[annual_obra['Year'] == year]['annual_Q_obra'].values[0]
    diferencia_mm = q_base_rep - q_obra_rep
    diferencia_m3ha = diferencia_mm * 10

    df['Evento_base'] = df['PCP'] > 0.2 * S_base
    df['Evento_obra'] = df['PCP'] > 0.2 * S_obra
    eventos_base_rep = df[(df['Year'] == year) & (df['Evento_base'])].shape[0]
    eventos_obra_rep = df[(df['Year'] == year) & (df['Evento_obra'])].shape[0]
    pp_anual = df[df['Year'] == year]['PCP'].sum()
    coef_q_base = q_base_rep / pp_anual if pp_anual > 0 else 0
    coef_q_obra = q_obra_rep / pp_anual if pp_anual > 0 else 0

    print(f"\nSitio: {nombre} ({lat:.4f}, {lon:.4f})")
    print(f"Mediana escurrimiento base (todos los años): {median_base:.2f} mm/año")
    print(f"Año seleccionado: {int(year)}")
    print(f"Escurrimiento base: {q_base_rep:.2f} mm")
    print(f"Escurrimiento con obras: {q_obra_rep:.2f} mm")
    print(f"Beneficio volumétrico: {diferencia_mm:.2f} mm   =   {diferencia_m3ha:.2f} m³/ha")
    print(f"Eventos con P > 0.2S base: {eventos_base_rep}")
    print(f"Eventos con P > 0.2S con obras: {eventos_obra_rep}")
    print(f"Coef. escurrimiento base: {coef_q_base:.2%}")
    print(f"Coef. escurrimiento con obras: {coef_q_obra:.2%}")

    df_rep = df[df['Year'] == year]
    plt.figure(figsize=(15,6))
    plt.plot(df_rep['Date'], df_rep['PCP'], color='navy', label='Precipitación (mm)', linewidth=1, alpha=0.6)
    plt.plot(df_rep['Date'], df_rep['Q_base'], color='red', label='Escurrimiento Base (mm)', linewidth=1)
    plt.plot(df_rep['Date'], df_rep['Q_obra'], color='green', label='Escurrimiento con obras (mm)', linewidth=1, linestyle='--')
    plt.title(f'Hidrograma año {int(year)} - {nombre}')
    plt.xlabel('Fecha')
    plt.ylabel('mm')
    plt.legend()
    plt.grid(alpha=0.3)
    plt.tight_layout()
    plt.show()

    plt.figure(figsize=(15,6))
    plt.bar('Base', q_base_rep, color='red', alpha=0.8)
    plt.bar('Con obras', q_obra_rep, color='green', alpha=0.8)
    plt.bar('Beneficio', diferencia_mm, color='blue', alpha=0.8)
    plt.ylabel('Escurrimiento anual (mm)')
    plt.title(f'Comparación de escurrimiento y beneficio volumétrico\nAño {int(year)} - {nombre}')
    plt.grid(axis='y', alpha=0.2)
    plt.tight_layout()
    plt.show()

# --- Widgets interactivos ---

nombre_default = "Toroto"
lat_default = 19.418922222134
lon_default = -99.16033686822948
CN_base_default = 90
CN_obra_default = 80

year_rep = get_data_and_yearrep(lat_default, lon_default, CN_base_default)[0]

interact(
        run_dashboard,
        nombre=Text(value=nombre_default, description="Nombre:"),
        lat=FloatText(value=lat_default, description="Latitud:"),
        lon=FloatText(value=lon_default, description="Longitud:"),
        CN_base=IntText(value=CN_base_default, description="CN base:"),
        CN_obra=IntText(value=CN_obra_default, description="CN obras:"),
        year=IntText(value=year_rep, description="Año")
    )

El sistema no puede encontrar la ruta especificada.


interactive(children=(Text(value='Toroto', description='Nombre:'), FloatText(value=19.418922222134, descriptio…

<function __main__.run_dashboard(nombre, lat, lon, CN_base, CN_obra, year)>

In [13]:
import pandas as pd
from pydaymet import get_bycoords
from datetime import date
import matplotlib.pyplot as plt
from ipywidgets import FloatText, IntText, Text, VBox, HBox, Button, Output

def obtener_datos(lat, lon, CN_base, CN_obra):
    # Descargar y procesar datos
    start = date(1980, 1, 1)
    end = date(date.today().year - 1, 12, 31)
    dates = (start.strftime("%Y-%m-%d"), end.strftime("%Y-%m-%d"))
    df = get_bycoords((lon, lat), dates, variables=['prcp'], time_scale='daily').reset_index()
    df = df.rename(columns={'time':'Date', 'prcp (mm/day)':'PCP'})[['Date', 'PCP']]
    df['PCP'] = df['PCP'].interpolate(method='linear', limit=1)
    df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d', errors='coerce')
    df = df.dropna(subset=['Date'])
    df['Year'] = df['Date'].dt.year

    # Escurrimiento
    S_base = 25400 / CN_base - 254
    S_obra = 25400 / CN_obra - 254
    def runoff(P, S):
        return ((P - 0.2*S)**2)/(P + 0.8*S) if P > 0.2*S else 0
    df['Q_base'] = df['PCP'].apply(lambda P: runoff(P, S_base))
    df['Q_obra'] = df['PCP'].apply(lambda P: runoff(P, S_obra))

    annual_base = df.groupby('Year')['Q_base'].sum().reset_index(name='annual_Q_base')
    annual_obra = df.groupby('Year')['Q_obra'].sum().reset_index(name='annual_Q_obra')

    # Mediana y año representativo
    median_base = annual_base['annual_Q_base'].median()
    annual_base['diff_mediana'] = (annual_base['annual_Q_base'] - median_base).abs()
    min_diff = annual_base['diff_mediana'].min()
    candidatos = annual_base[annual_base['diff_mediana'] == min_diff]
    year_rep = int(candidatos['Year'].max())
    return df, annual_base, annual_obra, median_base, year_rep

def run_dashboard(nombre, lat, lon, CN_base, CN_obra, year, output):
    output.clear_output()
    with output:
        df, annual_base, annual_obra, median_base, year_rep = obtener_datos(lat, lon, CN_base, CN_obra)

        # Validación de año
        if year not in annual_base['Year'].values:
            print(f"Año {year} no disponible. Elige entre {annual_base['Year'].min()} y {annual_base['Year'].max()}.")
            return

        # Métricas del año seleccionado
        q_base_rep = annual_base[annual_base['Year'] == year]['annual_Q_base'].values[0]
        q_obra_rep = annual_obra[annual_obra['Year'] == year]['annual_Q_obra'].values[0]
        diferencia_mm = q_base_rep - q_obra_rep
        diferencia_m3ha = diferencia_mm * 10

        df['Evento_base'] = df['PCP'] > 0.2 * (25400 / CN_base - 254)
        df['Evento_obra'] = df['PCP'] > 0.2 * (25400 / CN_obra - 254)
        eventos_base_rep = df[(df['Year'] == year) & (df['Evento_base'])].shape[0]
        eventos_obra_rep = df[(df['Year'] == year) & (df['Evento_obra'])].shape[0]
        pp_anual = df[df['Year'] == year]['PCP'].sum()
        coef_q_base = q_base_rep / pp_anual if pp_anual > 0 else 0
        coef_q_obra = q_obra_rep / pp_anual if pp_anual > 0 else 0

        print(f"\nSitio: {nombre} ({lat:.4f}, {lon:.4f})")
        print(f"Mediana escurrimiento base (todos los años): {median_base:.2f} mm/año")
        print(f"Año representativo: {year_rep}")
        print(f"Año seleccionado: {int(year)}")
        print(f"Escurrimiento base: {q_base_rep:.2f} mm")
        print(f"Escurrimiento con obras: {q_obra_rep:.2f} mm")
        print(f"Beneficio volumétrico: {diferencia_mm:.2f} mm   =   {diferencia_m3ha:.2f} m³/ha")
        print(f"Eventos con P > 0.2S base: {eventos_base_rep}")
        print(f"Eventos con P > 0.2S con obras: {eventos_obra_rep}")
        print(f"Coef. escurrimiento base: {coef_q_base:.2%}")
        print(f"Coef. escurrimiento con obras: {coef_q_obra:.2%}")

        # Hidrograma
        df_rep = df[df['Year'] == year]
        plt.figure(figsize=(15,6))
        plt.plot(df_rep['Date'], df_rep['PCP'], color='navy', label='Precipitación (mm)', linewidth=1, alpha=0.6)
        plt.plot(df_rep['Date'], df_rep['Q_base'], color='red', label='Escurrimiento Base (mm)', linewidth=1)
        plt.plot(df_rep['Date'], df_rep['Q_obra'], color='green', label='Escurrimiento con obras (mm)', linewidth=1, linestyle='--')
        plt.title(f'Hidrograma año {int(year)} - {nombre}')
        plt.xlabel('Fecha')
        plt.ylabel('mm')
        plt.legend()
        plt.grid(alpha=0.3)
        plt.tight_layout()
        plt.show()

        # Comparación barra
        plt.figure(figsize=(7,5))
        plt.bar('Base', q_base_rep, color='red', alpha=0.8)
        plt.bar('Con obras', q_obra_rep, color='green', alpha=0.8)
        plt.bar('Beneficio', diferencia_mm, color='blue', alpha=0.8)
        plt.ylabel('Escurrimiento anual (mm)')
        plt.title(f'Comparación de escurrimiento y beneficio volumétrico\nAño {int(year)} - {nombre}')
        plt.grid(axis='y', alpha=0.2)
        plt.tight_layout()
        plt.show()

# === Widgets ===
nombre_default = "Toroto"
lat_default = 19.418922222134
lon_default = -99.16033686822948
CN_base_default = 90
CN_obra_default = 80

nombre = Text(value=nombre_default, description="Nombre:")
lat = FloatText(value=lat_default, description="Latitud:")
lon = FloatText(value=lon_default, description="Longitud:")
CN_base = IntText(value=CN_base_default, description="CN base:")
CN_obra = IntText(value=CN_obra_default, description="CN obras:")

# Inicializa año representativo
_, annual_base0, _, _, year_rep0 = obtener_datos(lat_default, lon_default, CN_base_default, CN_obra_default)
year = IntText(value=year_rep0, description="Año")

btn_rep = Button(description="Calcular año representativo", button_style='info')
output = Output()

def actualizar_dashboard(change=None):
    run_dashboard(nombre.value, lat.value, lon.value, CN_base.value, CN_obra.value, year.value, output)

# Observa cambios
nombre.observe(actualizar_dashboard, names='value')
lat.observe(actualizar_dashboard, names='value')
lon.observe(actualizar_dashboard, names='value')
CN_base.observe(actualizar_dashboard, names='value')
CN_obra.observe(actualizar_dashboard, names='value')
year.observe(actualizar_dashboard, names='value')

def set_year_rep(b):
    _, annual_base, _, _, year_rep = obtener_datos(lat.value, lon.value, CN_base.value, CN_obra.value)
    year.value = year_rep

btn_rep.on_click(set_year_rep)

# Interfaz
from IPython.display import display
display(VBox([
    HBox([nombre, lat, lon]),
    HBox([CN_base, CN_obra, year, btn_rep]),
    output
]))
actualizar_dashboard()

In [15]:
import pandas as pd
from pydaymet import get_bycoords
from datetime import date
import matplotlib.pyplot as plt
import seaborn as sns
from ipywidgets import FloatText, IntText, Text, VBox, HBox, Button, Output

def obtener_datos(lat, lon, CN_base, CN_obra):
    # Descargar y procesar datos
    start = date(1980, 1, 1)
    end = date(date.today().year - 1, 12, 31)
    dates = (start.strftime("%Y-%m-%d"), end.strftime("%Y-%m-%d"))
    df = get_bycoords((lon, lat), dates, variables=['prcp'], time_scale='daily').reset_index()
    df = df.rename(columns={'time':'Date', 'prcp (mm/day)':'PCP'})[['Date', 'PCP']]
    df['PCP'] = df['PCP'].interpolate(method='linear', limit=1)
    df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d', errors='coerce')
    df = df.dropna(subset=['Date'])
    df['Year'] = df['Date'].dt.year

    # Escurrimiento
    S_base = 25400 / CN_base - 254
    S_obra = 25400 / CN_obra - 254
    def runoff(P, S):
        return ((P - 0.2*S)**2)/(P + 0.8*S) if P > 0.2*S else 0
    df['Q_base'] = df['PCP'].apply(lambda P: runoff(P, S_base))
    df['Q_obra'] = df['PCP'].apply(lambda P: runoff(P, S_obra))

    annual_base = df.groupby('Year')['Q_base'].sum().reset_index(name='annual_Q_base')
    annual_obra = df.groupby('Year')['Q_obra'].sum().reset_index(name='annual_Q_obra')

    # Mediana y año representativo
    median_base = annual_base['annual_Q_base'].median()
    annual_base['diff_mediana'] = (annual_base['annual_Q_base'] - median_base).abs()
    min_diff = annual_base['diff_mediana'].min()
    candidatos = annual_base[annual_base['diff_mediana'] == min_diff]
    year_rep = int(candidatos['Year'].max())
    return df, annual_base, annual_obra, median_base, year_rep

def run_dashboard(nombre, lat, lon, CN_base, CN_obra, year, output):
    output.clear_output()
    with output:
        df, annual_base, annual_obra, median_base, year_rep = obtener_datos(lat, lon, CN_base, CN_obra)

        # Validación de año
        if year not in annual_base['Year'].values:
            print(f"Año {year} no disponible. Elige entre {annual_base['Year'].min()} y {annual_base['Year'].max()}.")
            return

        # Métricas del año seleccionado
        q_base_rep = annual_base[annual_base['Year'] == year]['annual_Q_base'].values[0]
        q_obra_rep = annual_obra[annual_obra['Year'] == year]['annual_Q_obra'].values[0]
        diferencia_mm = q_base_rep - q_obra_rep
        diferencia_m3ha = diferencia_mm * 10

        df['Evento_base'] = df['PCP'] > 0.2 * (25400 / CN_base - 254)
        df['Evento_obra'] = df['PCP'] > 0.2 * (25400 / CN_obra - 254)
        eventos_base_rep = df[(df['Year'] == year) & (df['Evento_base'])].shape[0]
        eventos_obra_rep = df[(df['Year'] == year) & (df['Evento_obra'])].shape[0]
        pp_anual = df[df['Year'] == year]['PCP'].sum()
        coef_q_base = q_base_rep / pp_anual if pp_anual > 0 else 0
        coef_q_obra = q_obra_rep / pp_anual if pp_anual > 0 else 0

        print(f"\nSitio: {nombre} ({lat:.4f}, {lon:.4f})")
        print(f"Mediana escurrimiento base (todos los años): {median_base:.2f} mm/año")
        print(f"Año representativo: {year_rep}")
        print(f"Año seleccionado: {int(year)}")
        print(f"Escurrimiento base: {q_base_rep:.2f} mm")
        print(f"Escurrimiento con obras: {q_obra_rep:.2f} mm")
        print(f"Beneficio volumétrico: {diferencia_mm:.2f} mm   =   {diferencia_m3ha:.2f} m³/ha")
        print(f"Eventos con P > 0.2S base: {eventos_base_rep}")
        print(f"Eventos con P > 0.2S con obras: {eventos_obra_rep}")
        print(f"Coef. escurrimiento base: {coef_q_base:.2%}")
        print(f"Coef. escurrimiento con obras: {coef_q_obra:.2%}")

        # Hidrograma
        df_rep = df[df['Year'] == year]
        plt.figure(figsize=(15,6))
        plt.plot(df_rep['Date'], df_rep['PCP'], color='navy', label='Precipitación (mm)', linewidth=1, alpha=0.6)
        plt.plot(df_rep['Date'], df_rep['Q_base'], color='red', label='Escurrimiento Base (mm)', linewidth=1)
        plt.plot(df_rep['Date'], df_rep['Q_obra'], color='green', label='Escurrimiento con obras (mm)', linewidth=1, linestyle='--')
        plt.title(f'Hidrograma año {int(year)} - {nombre}')
        plt.xlabel('Fecha')
        plt.ylabel('mm')
        plt.legend()
        plt.grid(alpha=0.3)
        plt.tight_layout()
        plt.show()

        # Comparación barra
        plt.figure(figsize=(7,5))
        plt.bar('Base', q_base_rep, color='red', alpha=0.8)
        plt.bar('Con obras', q_obra_rep, color='green', alpha=0.8)
        plt.bar('Beneficio', diferencia_mm, color='blue', alpha=0.8)
        plt.ylabel('Escurrimiento anual (mm)')
        plt.title(f'Comparación de escurrimiento y beneficio volumétrico\nAño {int(year)} - {nombre}')
        plt.grid(axis='y', alpha=0.2)
        plt.tight_layout()
        plt.show()

        # Boxplot de escurrimiento anual base
        plt.figure(figsize=(9,2))
        plt.boxplot(annual_base['annual_Q_base'], vert=False, patch_artist=True,
                    boxprops=dict(facecolor="lightblue"))
        plt.scatter(median_base, 1, color='blue', label='Mediana', zorder=3)
        plt.scatter(annual_base[annual_base['Year']==year_rep]['annual_Q_base'], 1, color='orange', label='Año representativo', zorder=3)
        plt.scatter(annual_base[annual_base['Year']==year]['annual_Q_base'], 1, color='green', label='Año seleccionado', zorder=3)
        plt.xlabel('Escurrimiento anual base (mm)')
        plt.yticks([])
        plt.legend()
        plt.title("Distribución escurrimiento anual base (Boxplot)")
        plt.tight_layout()
        plt.show()

        # Histograma con KDE
        plt.figure(figsize=(9,4))
        sns.histplot(annual_base['annual_Q_base'], kde=True, color='gray', alpha=0.5, label="Todos los años")
        plt.axvline(median_base, color='blue', linestyle='--', label='Mediana')
        plt.scatter(annual_base[annual_base['Year']==year_rep]['annual_Q_base'], 0, color='orange', label='Año representativo', s=100, zorder=3)
        plt.scatter(annual_base[annual_base['Year']==year]['annual_Q_base'], 0, color='green', label='Año seleccionado', s=100, zorder=3)
        plt.xlabel("Escurrimiento anual base (mm)")
        plt.ylabel("Frecuencia")
        plt.title("Histograma escurrimiento anual base (KDE)")
        plt.legend()
        plt.tight_layout()
        plt.show()

# === Widgets ===
nombre_default = "Toroto"
lat_default = 19.418922222134
lon_default = -99.16033686822948
CN_base_default = 90
CN_obra_default = 80

nombre = Text(value=nombre_default, description="Nombre:")
lat = FloatText(value=lat_default, description="Latitud:")
lon = FloatText(value=lon_default, description="Longitud:")
CN_base = IntText(value=CN_base_default, description="CN base:")
CN_obra = IntText(value=CN_obra_default, description="CN obras:")

# Inicializa año representativo
_, annual_base0, _, _, year_rep0 = obtener_datos(lat_default, lon_default, CN_base_default, CN_obra_default)
year = IntText(value=year_rep0, description="Año")

btn_rep = Button(description="Calcular año representativo", button_style='info')
output = Output()

def actualizar_dashboard(change=None):
    run_dashboard(nombre.value, lat.value, lon.value, CN_base.value, CN_obra.value, year.value, output)

# Observa cambios
nombre.observe(actualizar_dashboard, names='value')
lat.observe(actualizar_dashboard, names='value')
lon.observe(actualizar_dashboard, names='value')
CN_base.observe(actualizar_dashboard, names='value')
CN_obra.observe(actualizar_dashboard, names='value')
year.observe(actualizar_dashboard, names='value')

def set_year_rep(b):
    _, annual_base, _, _, year_rep = obtener_datos(lat.value, lon.value, CN_base.value, CN_obra.value)
    year.value = year_rep

btn_rep.on_click(set_year_rep)

# Interfaz
from IPython.display import display
display(VBox([
    HBox([nombre, lat, lon]),
    HBox([CN_base, CN_obra, year, btn_rep]),
    output
]))
actualizar_dashboard()

In [2]:
# @title
# Instala dependencias (ejecuta sólo una vez por sesión)
!pip install pandas pydaymet datetime matplotlib ipywidgets > /dev/null

import pandas as pd
from pydaymet import get_bycoords
from datetime import date
import matplotlib.pyplot as plt
from ipywidgets import FloatText, IntText, Text, IntSlider, Button, HBox, VBox, Output, SelectMultiple
from IPython.display import display

def get_data_and_yearrep(lat, lon, CN_base):
    start = date(1980, 1, 1)
    end = date(date.today().year - 1, 12, 31)
    dates = (start.strftime("%Y-%m-%d"), end.strftime("%Y-%m-%d"))
    df = get_bycoords((lon, lat), dates, variables=['prcp'], time_scale='daily').reset_index()
    df = df.rename(columns={'time':'Date', 'prcp (mm/day)':'PCP'})[['Date', 'PCP']]
    df['PCP'] = df['PCP'].interpolate(method='linear', limit=1)
    df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d', errors='coerce')
    df = df.dropna(subset=['Date'])
    df['Year'] = df['Date'].dt.year
    df['Month'] = df['Date'].dt.month

    S_base = 25400 / CN_base - 254
    def runoff(P, S):
        return ((P - 0.2*S)**2)/(P + 0.8*S) if P > 0.2*S else 0
    df['Q_base'] = df['PCP'].apply(lambda P: runoff(P, S_base))
    annual_base = df.groupby('Year')['Q_base'].sum().reset_index(name='annual_Q_base')
    median_base = annual_base['annual_Q_base'].median()
    annual_base['diff_mediana'] = (annual_base['annual_Q_base'] - median_base).abs()
    min_diff = annual_base['diff_mediana'].min()
    candidatos = annual_base[annual_base['diff_mediana'] == min_diff]
    year_rep = candidatos['Year'].max()
    return int(year_rep), int(df['Year'].min()), int(df['Year'].max()), df

def run_dashboard(nombre, lat, lon, CN_base, CN_obra, year, mes_ini, mes_fin, variables, output):
    output.clear_output()
    with output:
        year_rep, min_year, max_year, df = get_data_and_yearrep(lat, lon, CN_base)
        if year < min_year or year > max_year:
            print(f"Año {year} no disponible. Elige entre {min_year} y {max_year}.")
            return

        S_base = 25400 / CN_base - 254
        S_obra = 25400 / CN_obra - 254
        def runoff(P, S):
            return ((P - 0.2*S)**2)/(P + 0.8*S) if P > 0.2*S else 0
        df['Q_base'] = df['PCP'].apply(lambda P: runoff(P, S_base))
        df['Q_obra'] = df['PCP'].apply(lambda P: runoff(P, S_obra))
        df['Evento_base'] = df['PCP'] > 0.2 * S_base
        df['Evento_obra'] = df['PCP'] > 0.2 * S_obra

        # --- FILTRA RANGO DE MESES EN EL AÑO SELECCIONADO ---
        df_rep = df[(df['Year'] == year) & (df['Month'] >= mes_ini) & (df['Month'] <= mes_fin)]

        # Métricas del rango de meses (solo para el año seleccionado)
        q_base_rep = df_rep['Q_base'].sum()
        q_obra_rep = df_rep['Q_obra'].sum()
        diferencia_mm = q_base_rep - q_obra_rep
        diferencia_m3ha = diferencia_mm * 10
        eventos_base_rep = df_rep['Evento_base'].sum()
        eventos_obra_rep = df_rep['Evento_obra'].sum()
        pp_rango = df_rep['PCP'].sum()
        coef_q_base = q_base_rep / pp_rango if pp_rango > 0 else 0
        coef_q_obra = q_obra_rep / pp_rango if pp_rango > 0 else 0

        print(f"\nSitio: {nombre} ({lat:.4f}, {lon:.4f})")
        print(f"Año representativo (completo): {year_rep}")
        print(f"Año seleccionado: {int(year)}")
        print(f"Meses mostrados: {mes_ini}-{mes_fin}")
        print(f"Escurrimiento base (rango): {q_base_rep:.2f} mm")
        print(f"Escurrimiento con obras (rango): {q_obra_rep:.2f} mm")
        print(f"Beneficio volumétrico (rango): {diferencia_mm:.2f} mm   =   {diferencia_m3ha:.2f} m³/ha")
        print(f"Eventos con P > 0.2S base (rango): {int(eventos_base_rep)}")
        print(f"Eventos con P > 0.2S con obras (rango): {int(eventos_obra_rep)}")
        print(f"Coef. escurrimiento base: {coef_q_base:.2%}")
        print(f"Coef. escurrimiento con obras: {coef_q_obra:.2%}")

        # --- HIDROGRAMA SOLO DEL RANGO DE MESES Y VARIABLES SELECCIONADAS ---
        plt.figure(figsize=(15,6))
        if 'Precipitación' in variables:
            plt.plot(df_rep['Date'], df_rep['PCP'], color='navy', label='Precipitación (mm)', linewidth=1, alpha=0.6)
        if 'Escurrimiento base' in variables:
            plt.plot(df_rep['Date'], df_rep['Q_base'], color='red', label='Escurrimiento Base (mm)', linewidth=1)
        if 'Escurrimiento con obras' in variables:
            plt.plot(df_rep['Date'], df_rep['Q_obra'], color='green', label='Escurrimiento con obras (mm)', linewidth=1, linestyle='--')
        plt.title(f'Hidrograma año {int(year)}, meses {mes_ini}-{mes_fin} - {nombre}')
        plt.xlabel('Fecha')
        plt.ylabel('mm')
        plt.legend()
        plt.grid(alpha=0.3)
        plt.tight_layout()
        plt.show()

        # Comparación barra para el rango de meses
        plt.figure(figsize=(10,5))
        plt.bar(['Base','Con obras','Beneficio'], [q_base_rep, q_obra_rep, diferencia_mm], color=['red','green','blue'], alpha=0.8)
        plt.ylabel('Escurrimiento (mm) en rango')
        plt.title(f'Comparación escurrimiento y beneficio\nAño {int(year)}, meses {mes_ini}-{mes_fin} - {nombre}')
        plt.grid(axis='y', alpha=0.2)
        plt.tight_layout()
        plt.show()

        # Guarda los datos del hidrograma mostrado
        global last_df_export
        # Solo las variables seleccionadas más la fecha
        export_cols = ['Date']
        if 'Precipitación' in variables:
            export_cols.append('PCP')
        if 'Escurrimiento base' in variables:
            export_cols.append('Q_base')
        if 'Escurrimiento con obras' in variables:
            export_cols.append('Q_obra')
        last_df_export = df_rep[export_cols].copy()

# --- Widgets interactivos ---
nombre_default = "Toroto"
lat_default = 19.418922222134
lon_default = -99.16033686822948
CN_base_default = 90
CN_obra_default = 80

year_rep, min_year, max_year, df_init = get_data_and_yearrep(lat_default, lon_default, CN_base_default)
nombre = Text(value=nombre_default, description="Nombre:")
lat = FloatText(value=lat_default, description="Latitud:")
lon = FloatText(value=lon_default, description="Longitud:")
CN_base = IntText(value=CN_base_default, description="CN base:")
CN_obra = IntText(value=CN_obra_default, description="CN obras:")
year = IntText(value=year_rep, description="Año")
mes_ini = IntSlider(value=1, min=1, max=12, description="Mes inicial:", continuous_update=False)
mes_fin = IntSlider(value=12, min=1, max=12, description="Mes final:", continuous_update=False)
variables_widget = SelectMultiple(
    options=['Precipitación', 'Escurrimiento base', 'Escurrimiento con obras'],
    value=('Precipitación', 'Escurrimiento base', 'Escurrimiento con obras'),
    description='Variables:', style={'description_width':'initial'}
)
btn_rep = Button(description="Año representativo", button_style='info')
btn_export = Button(description="Exportar datos mostrados", button_style='success')
output = Output()

last_df_export = None  # Global para exportación

def actualizar_dashboard(change=None):
    # Ajusta meses para evitar errores (ej: mes_ini > mes_fin)
    if mes_ini.value > mes_fin.value:
        mes_ini.value, mes_fin.value = mes_fin.value, mes_ini.value
    run_dashboard(
        nombre.value, lat.value, lon.value, CN_base.value, CN_obra.value,
        year.value, mes_ini.value, mes_fin.value,
        list(variables_widget.value), output
    )

for w in [nombre, lat, lon, CN_base, CN_obra, year, mes_ini, mes_fin, variables_widget]:
    w.observe(actualizar_dashboard, names='value')

def set_year_rep(b):
    year_rep, _, _, _ = get_data_and_yearrep(lat.value, lon.value, CN_base.value)
    year.value = year_rep

btn_rep.on_click(set_year_rep)

def exportar_datos(b):
    global last_df_export
    if last_df_export is not None and len(last_df_export) > 0:
        fname = f"{nombre.value}_{year.value}_meses_{mes_ini.value}-{mes_fin.value}.csv"
        last_df_export.to_csv(fname, index=False)
        print(f"Archivo exportado: {fname}")
    else:
        print("No hay datos para exportar. Cambia los parámetros y vuelve a intentar.")

btn_export.on_click(exportar_datos)

# Interfaz
display(VBox([
    HBox([nombre, lat, lon]),
    HBox([CN_base, CN_obra, year, btn_rep]),
    HBox([mes_ini, mes_fin]),
    variables_widget,
    btn_export,
    output
]))
actualizar_dashboard()

El sistema no puede encontrar la ruta especificada.


In [19]:
# @title
# Instala dependencias (si no las tienes) - ejecuta sólo una vez por sesión
!pip install pandas pydaymet datetime matplotlib ipywidgets > /dev/null

import pandas as pd
from pydaymet import get_bycoords
from datetime import date
import matplotlib.pyplot as plt
from ipywidgets import FloatText, IntText, Text, IntSlider, Button, HBox, VBox, Output, Checkbox
from IPython.display import display

def get_data_and_yearrep(lat, lon, CN_base):
    start = date(1980, 1, 1)
    end = date(date.today().year - 1, 12, 31)
    dates = (start.strftime("%Y-%m-%d"), end.strftime("%Y-%m-%d"))
    df = get_bycoords((lon, lat), dates, variables=['prcp'], time_scale='daily').reset_index()
    df = df.rename(columns={'time':'Date', 'prcp (mm/day)':'PCP'})[['Date', 'PCP']]
    df['PCP'] = df['PCP'].interpolate(method='linear', limit=1)
    df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d', errors='coerce')
    df = df.dropna(subset=['Date'])
    df['Year'] = df['Date'].dt.year
    df['Month'] = df['Date'].dt.month

    S_base = 25400 / CN_base - 254
    def runoff(P, S):
        return ((P - 0.2*S)**2)/(P + 0.8*S) if P > 0.2*S else 0
    df['Q_base'] = df['PCP'].apply(lambda P: runoff(P, S_base))
    annual_base = df.groupby('Year')['Q_base'].sum().reset_index(name='annual_Q_base')
    median_base = annual_base['annual_Q_base'].median()
    annual_base['diff_mediana'] = (annual_base['annual_Q_base'] - median_base).abs()
    min_diff = annual_base['diff_mediana'].min()
    candidatos = annual_base[annual_base['diff_mediana'] == min_diff]
    year_rep = candidatos['Year'].max()
    return int(year_rep), int(df['Year'].min()), int(df['Year'].max()), df

def run_dashboard(nombre, lat, lon, CN_base, CN_obra, year, mes_ini, mes_fin, variables, output):
    output.clear_output()
    with output:
        year_rep, min_year, max_year, df = get_data_and_yearrep(lat, lon, CN_base)
        if year < min_year or year > max_year:
            print(f"Año {year} no disponible. Elige entre {min_year} y {max_year}.")
            return

        S_base = 25400 / CN_base - 254
        S_obra = 25400 / CN_obra - 254
        def runoff(P, S):
            return ((P - 0.2*S)**2)/(P + 0.8*S) if P > 0.2*S else 0
        df['Q_base'] = df['PCP'].apply(lambda P: runoff(P, S_base))
        df['Q_obra'] = df['PCP'].apply(lambda P: runoff(P, S_obra))
        df['Evento_base'] = df['PCP'] > 0.2 * S_base
        df['Evento_obra'] = df['PCP'] > 0.2 * S_obra

        # --- FILTRA RANGO DE MESES EN EL AÑO SELECCIONADO ---
        df_rep = df[(df['Year'] == year) & (df['Month'] >= mes_ini) & (df['Month'] <= mes_fin)]

        # Métricas del rango de meses (solo para el año seleccionado)
        q_base_rep = df_rep['Q_base'].sum()
        q_obra_rep = df_rep['Q_obra'].sum()
        diferencia_mm = q_base_rep - q_obra_rep
        diferencia_m3ha = diferencia_mm * 10
        eventos_base_rep = df_rep['Evento_base'].sum()
        eventos_obra_rep = df_rep['Evento_obra'].sum()
        pp_rango = df_rep['PCP'].sum()
        coef_q_base = q_base_rep / pp_rango if pp_rango > 0 else 0
        coef_q_obra = q_obra_rep / pp_rango if pp_rango > 0 else 0

        print(f"\nSitio: {nombre} ({lat:.4f}, {lon:.4f})")
        print(f"Año representativo (completo): {year_rep}")
        print(f"Año seleccionado: {int(year)}")
        print(f"Meses mostrados: {mes_ini}-{mes_fin}")
        print(f"Escurrimiento base (rango): {q_base_rep:.2f} mm")
        print(f"Escurrimiento con obras (rango): {q_obra_rep:.2f} mm")
        print(f"Beneficio volumétrico (rango): {diferencia_mm:.2f} mm   =   {diferencia_m3ha:.2f} m³/ha")
        print(f"Eventos con P > 0.2S base (rango): {int(eventos_base_rep)}")
        print(f"Eventos con P > 0.2S con obras (rango): {int(eventos_obra_rep)}")
        print(f"Coef. escurrimiento base: {coef_q_base:.2%}")
        print(f"Coef. escurrimiento con obras: {coef_q_obra:.2%}")

        # --- HIDROGRAMA SOLO DEL RANGO DE MESES Y VARIABLES SELECCIONADAS ---
        plt.figure(figsize=(15,6))
        if 'Precipitación' in variables:
            plt.plot(df_rep['Date'], df_rep['PCP'], color='navy', label='Precipitación (mm)', linewidth=1, alpha=0.6)
        if 'Escurrimiento base' in variables:
            plt.plot(df_rep['Date'], df_rep['Q_base'], color='red', label='Escurrimiento Base (mm)', linewidth=1)
        if 'Escurrimiento con obras' in variables:
            plt.plot(df_rep['Date'], df_rep['Q_obra'], color='green', label='Escurrimiento con obras (mm)', linewidth=1, linestyle='--')
        plt.title(f'Hidrograma año {int(year)}- {nombre}')
        plt.xlabel('Fecha')
        plt.ylabel('mm')
        plt.legend()
        plt.grid(alpha=0.3)
        plt.tight_layout()
        plt.show()

        # Comparación barra para el rango de meses
        plt.figure(figsize=(10,5))
        plt.bar(['Base','Con obras','Beneficio'], [q_base_rep, q_obra_rep, diferencia_mm], color=['red','green','blue'], alpha=0.8)
        plt.ylabel('Escurrimiento (mm) en rango')
        plt.title(f'Comparación escurrimiento y beneficio\nAño {int(year)}, meses {mes_ini}-{mes_fin} - {nombre}')
        plt.grid(axis='y', alpha=0.2)
        plt.tight_layout()
        plt.show()

        # Guarda los datos del hidrograma mostrado
        global last_df_export
        export_cols = ['Date']
        if 'Precipitación' in variables:
            export_cols.append('PCP')
        if 'Escurrimiento base' in variables:
            export_cols.append('Q_base')
        if 'Escurrimiento con obras' in variables:
            export_cols.append('Q_obra')
        last_df_export = df_rep[export_cols].copy()

# --- Widgets interactivos ---
nombre_default = "Toroto"
lat_default = 19.418922222134
lon_default = -99.16033686822948
CN_base_default = 90
CN_obra_default = 80

year_rep, min_year, max_year, df_init = get_data_and_yearrep(lat_default, lon_default, CN_base_default)
nombre = Text(value=nombre_default, description="Nombre:")
lat = FloatText(value=lat_default, description="Latitud:")
lon = FloatText(value=lon_default, description="Longitud:")
CN_base = IntText(value=CN_base_default, description="CN base:")
CN_obra = IntText(value=CN_obra_default, description="CN obras:")
year = IntText(value=year_rep, description="Año")
mes_ini = IntSlider(value=1, min=1, max=12, description="Mes inicial:", continuous_update=False)
mes_fin = IntSlider(value=12, min=1, max=12, description="Mes final:", continuous_update=False)
prec_widget = Checkbox(value=True, description='Precipitación')
base_widget = Checkbox(value=True, description='Escurrimiento base')
obra_widget = Checkbox(value=True, description='Escurrimiento con obras')
btn_rep = Button(description="Año representativo", button_style='info')
btn_export = Button(description="Exportar datos mostrados", button_style='success')
output = Output()

last_df_export = None  # Global para exportación

def get_variables_selected():
    v = []
    if prec_widget.value:
        v.append('Precipitación')
    if base_widget.value:
        v.append('Escurrimiento base')
    if obra_widget.value:
        v.append('Escurrimiento con obras')
    return v

def actualizar_dashboard(change=None):
    if mes_ini.value > mes_fin.value:
        mes_ini.value, mes_fin.value = mes_fin.value, mes_ini.value
    run_dashboard(
        nombre.value, lat.value, lon.value, CN_base.value, CN_obra.value,
        year.value, mes_ini.value, mes_fin.value,
        get_variables_selected(), output
    )

for w in [nombre, lat, lon, CN_base, CN_obra, year, mes_ini, mes_fin, prec_widget, base_widget, obra_widget]:
    w.observe(actualizar_dashboard, names='value')

def set_year_rep(b):
    year_rep, _, _, _ = get_data_and_yearrep(lat.value, lon.value, CN_base.value)
    year.value = year_rep

btn_rep.on_click(set_year_rep)

def exportar_datos(b):
    global last_df_export
    if last_df_export is not None and len(last_df_export) > 0:
        fname = f"{nombre.value}_hidrograma_{year.value}_meses_{mes_ini.value}-{mes_fin.value}.csv"
        last_df_export.to_csv(fname, index=False)
        print(f"Archivo exportado: {fname}")
    else:
        print("No hay datos para exportar. Cambia los parámetros y vuelve a intentar.")

btn_export.on_click(exportar_datos)

# Interfaz
display(VBox([
    HBox([nombre, lat, lon]),
    HBox([CN_base, CN_obra, year, btn_rep]),
    HBox([mes_ini, mes_fin]),
    HBox([prec_widget, base_widget, obra_widget]),
    btn_export,
    output
]))
actualizar_dashboard()

El sistema no puede encontrar la ruta especificada.


In [1]:
# @title
# Instala dependencias (si no las tienes) - ejecuta sólo una vez por sesión
!pip install pandas pydaymet datetime matplotlib ipywidgets > /dev/null

import pandas as pd
from pydaymet import get_bycoords
from datetime import date
import matplotlib.pyplot as plt
from ipywidgets import FloatText, IntText, Text, IntSlider, Button, HBox, VBox, Output, Checkbox
from IPython.display import display

def get_data_and_yearrep(lat, lon, CN_base):
    start = date(1980, 1, 1)
    end = date(date.today().year - 1, 12, 31)
    dates = (start.strftime("%Y-%m-%d"), end.strftime("%Y-%m-%d"))
    df = get_bycoords((lon, lat), dates, variables=['prcp'], time_scale='daily').reset_index()
    df = df.rename(columns={'time':'Date', 'prcp (mm/day)':'PCP'})[['Date', 'PCP']]
    df['PCP'] = df['PCP'].interpolate(method='linear', limit=1)
    df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d', errors='coerce')
    df = df.dropna(subset=['Date'])
    df['Year'] = df['Date'].dt.year
    df['Month'] = df['Date'].dt.month

    S_base = 25400 / CN_base - 254
    def runoff(P, S):
        return ((P - 0.2*S)**2)/(P + 0.8*S) if P > 0.2*S else 0
    df['Q_base'] = df['PCP'].apply(lambda P: runoff(P, S_base))
    annual_base = df.groupby('Year')['Q_base'].sum().reset_index(name='annual_Q_base')
    median_base = annual_base['annual_Q_base'].median()
    annual_base['diff_mediana'] = (annual_base['annual_Q_base'] - median_base).abs()
    min_diff = annual_base['diff_mediana'].min()
    candidatos = annual_base[annual_base['diff_mediana'] == min_diff]
    year_rep = candidatos['Year'].max()
    return int(year_rep), int(df['Year'].min()), int(df['Year'].max()), df

def run_dashboard(nombre, lat, lon, CN_base, CN_obra, year, mes_ini, mes_fin, variables, output):
    output.clear_output()
    with output:
        year_rep, min_year, max_year, df = get_data_and_yearrep(lat, lon, CN_base)
        if year < min_year or year > max_year:
            print(f"Año {year} no disponible. Elige entre {min_year} y {max_year}.")
            return

        S_base = 25400 / CN_base - 254
        S_obra = 25400 / CN_obra - 254
        def runoff(P, S):
            return ((P - 0.2*S)**2)/(P + 0.8*S) if P > 0.2*S else 0
        df['Q_base'] = df['PCP'].apply(lambda P: runoff(P, S_base))
        df['Q_obra'] = df['PCP'].apply(lambda P: runoff(P, S_obra))
        df['Evento_base'] = df['PCP'] > 0.2 * S_base
        df['Evento_obra'] = df['PCP'] > 0.2 * S_obra

        # --- Datos año completo para métricas y barras ---
        df_anno = df[df['Year'] == year]

        # Métricas del año completo
        q_base_year = df_anno['Q_base'].sum()
        q_obra_year = df_anno['Q_obra'].sum()
        diferencia_mm = q_base_year - q_obra_year
        diferencia_m3ha = diferencia_mm * 10
        eventos_base_year = df_anno['Evento_base'].sum()
        eventos_obra_year = df_anno['Evento_obra'].sum()
        pp_anual = df_anno['PCP'].sum()
        coef_q_base = q_base_year / pp_anual if pp_anual > 0 else 0
        coef_q_obra = q_obra_year / pp_anual if pp_anual > 0 else 0

        print(f"\nSitio: {nombre} ({lat:.4f}, {lon:.4f})")
        print(f"Año representativo (completo): {year_rep}")
        print(f"Año seleccionado (completo): {int(year)}")
        print(f"Escurrimiento base (año completo): {q_base_year:.2f} mm")
        print(f"Escurrimiento con obras (año completo): {q_obra_year:.2f} mm")
        print(f"Beneficio volumétrico (año completo): {diferencia_mm:.2f} mm   =   {diferencia_m3ha:.2f} m³/ha")
        print(f"Eventos con P > 0.2S base (año completo): {int(eventos_base_year)}")
        print(f"Eventos con P > 0.2S con obras (año completo): {int(eventos_obra_year)}")
        print(f"Coef. escurrimiento base: {coef_q_base:.2%}")
        print(f"Coef. escurrimiento con obras: {coef_q_obra:.2%}")

        # --- HIDROGRAMA SOLO DEL RANGO DE MESES Y VARIABLES SELECCIONADAS ---
        df_rep = df_anno[(df_anno['Month'] >= mes_ini) & (df_anno['Month'] <= mes_fin)]

        plt.figure(figsize=(15,6))
        if 'Precipitación' in variables:
            plt.plot(df_rep['Date'], df_rep['PCP'], color='navy', label='Precipitación (mm)', linewidth=1, alpha=0.6)
        if 'Escurrimiento base' in variables:
            plt.plot(df_rep['Date'], df_rep['Q_base'], color='red', label='Escurrimiento Base (mm)', linewidth=1)
        if 'Escurrimiento con obras' in variables:
            plt.plot(df_rep['Date'], df_rep['Q_obra'], color='green', label='Escurrimiento con obras (mm)', linewidth=1, linestyle='--')
        plt.title(f'Hidrograma año {int(year)}, meses {mes_ini}-{mes_fin} - {nombre}')
        plt.xlabel('Fecha')
        plt.ylabel('mm')
        plt.legend()
        plt.grid(alpha=0.3)
        plt.tight_layout()
        plt.show()

        # Comparación barra (del año completo)
        plt.figure(figsize=(10,5))
        plt.bar(['Base','Con obras','Beneficio'], [q_base_year, q_obra_year, diferencia_mm], color=['red','green','blue'], alpha=0.8)
        plt.ylabel('Escurrimiento anual (mm)')
        plt.title(f'Comparación escurrimiento y beneficio volumétrico\nAño {int(year)} - {nombre}')
        plt.grid(axis='y', alpha=0.2)
        plt.tight_layout()
        plt.show()

        # Exporta solo lo mostrado en el hidrograma
        global last_df_export
        export_cols = ['Date']
        if 'Precipitación' in variables:
            export_cols.append('PCP')
        if 'Escurrimiento base' in variables:
            export_cols.append('Q_base')
        if 'Escurrimiento con obras' in variables:
            export_cols.append('Q_obra')
        last_df_export = df_rep[export_cols].copy()

# --- Widgets interactivos ---
nombre_default = "Toroto"
lat_default = 19.418922222134
lon_default = -99.16033686822948
CN_base_default = 90
CN_obra_default = 80

year_rep, min_year, max_year, df_init = get_data_and_yearrep(lat_default, lon_default, CN_base_default)
nombre = Text(value=nombre_default, description="Nombre:")
lat = FloatText(value=lat_default, description="Latitud:")
lon = FloatText(value=lon_default, description="Longitud:")
CN_base = IntText(value=CN_base_default, description="CN base:")
CN_obra = IntText(value=CN_obra_default, description="CN obras:")
year = IntText(value=year_rep, description="Año")
mes_ini = IntSlider(value=1, min=1, max=12, description="Mes inicial:", continuous_update=False)
mes_fin = IntSlider(value=12, min=1, max=12, description="Mes final:", continuous_update=False)
prec_widget = Checkbox(value=True, description='Precipitación')
base_widget = Checkbox(value=True, description='Escurrimiento base')
obra_widget = Checkbox(value=True, description='Escurrimiento con obras')
btn_rep = Button(description="Año representativo", button_style='info')
btn_export = Button(description="Exportar datos mostrados", button_style='success')
output = Output()

last_df_export = None  # Global para exportación

def get_variables_selected():
    v = []
    if prec_widget.value:
        v.append('Precipitación')
    if base_widget.value:
        v.append('Escurrimiento base')
    if obra_widget.value:
        v.append('Escurrimiento con obras')
    return v

def actualizar_dashboard(change=None):
    if mes_ini.value > mes_fin.value:
        mes_ini.value, mes_fin.value = mes_fin.value, mes_ini.value
    run_dashboard(
        nombre.value, lat.value, lon.value, CN_base.value, CN_obra.value,
        year.value, mes_ini.value, mes_fin.value,
        get_variables_selected(), output
    )

for w in [nombre, lat, lon, CN_base, CN_obra, year, mes_ini, mes_fin, prec_widget, base_widget, obra_widget]:
    w.observe(actualizar_dashboard, names='value')

def set_year_rep(b):
    year_rep, _, _, _ = get_data_and_yearrep(lat.value, lon.value, CN_base.value)
    year.value = year_rep

btn_rep.on_click(set_year_rep)

def exportar_datos(b):
    global last_df_export
    if last_df_export is not None and len(last_df_export) > 0:
        fname = f"{nombre.value}_hidrograma_{year.value}_meses_{mes_ini.value}-{mes_fin.value}.csv"
        last_df_export.to_csv(fname, index=False)
        print(f"Archivo exportado: {fname}")
    else:
        print("No hay datos para exportar. Cambia los parámetros y vuelve a intentar.")

btn_export.on_click(exportar_datos)

# Interfaz
display(VBox([
    HBox([nombre, lat, lon]),
    HBox([CN_base, CN_obra, year, btn_rep]),
    HBox([mes_ini, mes_fin]),
    HBox([prec_widget, base_widget, obra_widget]),
    btn_export,
    output
]))
actualizar_dashboard()

El sistema no puede encontrar la ruta especificada.


In [3]:
# @title
# Instala dependencias (si no las tienes) - ejecuta sólo una vez por sesión
!pip install pandas pydaymet datetime matplotlib ipywidgets > /dev/null

import pandas as pd
from pydaymet import get_bycoords
from datetime import date
import matplotlib.pyplot as plt
from ipywidgets import FloatText, IntText, Text, IntSlider, Button, HBox, VBox, Output, Checkbox
from IPython.display import display

def get_data_and_yearrep(lat, lon, CN_base):
    start = date(1980, 1, 1)
    end = date(date.today().year - 1, 12, 31)
    dates = (start.strftime("%Y-%m-%d"), end.strftime("%Y-%m-%d"))
    df = get_bycoords((lon, lat), dates, variables=['prcp'], time_scale='daily').reset_index()
    df = df.rename(columns={'time':'Date', 'prcp (mm/day)':'PCP'})[['Date', 'PCP']]
    df['PCP'] = df['PCP'].interpolate(method='linear', limit=1)
    df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d', errors='coerce')
    df = df.dropna(subset=['Date'])
    df['Year'] = df['Date'].dt.year
    df['Month'] = df['Date'].dt.month

    S_base = 25400 / CN_base - 254
    def runoff(P, S):
        return ((P - 0.2*S)**2)/(P + 0.8*S) if P > 0.2*S else 0
    df['Q_base'] = df['PCP'].apply(lambda P: runoff(P, S_base))
    annual_base = df.groupby('Year')['Q_base'].sum().reset_index(name='annual_Q_base')
    median_base = annual_base['annual_Q_base'].median()
    annual_base['diff_mediana'] = (annual_base['annual_Q_base'] - median_base).abs()
    min_diff = annual_base['diff_mediana'].min()
    candidatos = annual_base[annual_base['diff_mediana'] == min_diff]
    year_rep = candidatos['Year'].max()
    return int(year_rep), int(df['Year'].min()), int(df['Year'].max()), df

def run_dashboard(nombre, lat, lon, CN_base, CN_obra, year, mes_ini, mes_fin, variables, output):
    output.clear_output()
    with output:
        year_rep, min_year, max_year, df = get_data_and_yearrep(lat, lon, CN_base)
        if year < min_year or year > max_year:
            print(f"Año {year} no disponible. Elige entre {min_year} y {max_year}.")
            return

        S_base = 25400 / CN_base - 254
        S_obra = 25400 / CN_obra - 254
        def runoff(P, S):
            return ((P - 0.2*S)**2)/(P + 0.8*S) if P > 0.2*S else 0
        df['Q_base'] = df['PCP'].apply(lambda P: runoff(P, S_base))
        df['Q_obra'] = df['PCP'].apply(lambda P: runoff(P, S_obra))
        df['Evento_base'] = df['PCP'] > 0.2 * S_base
        df['Evento_obra'] = df['PCP'] > 0.2 * S_obra

        # --- Datos año completo para métricas y barras ---
        df_anno = df[df['Year'] == year]

        # Métricas del año completo
        q_base_year = df_anno['Q_base'].sum()
        q_obra_year = df_anno['Q_obra'].sum()
        diferencia_mm = q_base_year - q_obra_year
        diferencia_m3ha = diferencia_mm * 10
        eventos_base_year = df_anno['Evento_base'].sum()
        eventos_obra_year = df_anno['Evento_obra'].sum()
        pp_anual = df_anno['PCP'].sum()
        coef_q_base = q_base_year / pp_anual if pp_anual > 0 else 0
        coef_q_obra = q_obra_year / pp_anual if pp_anual > 0 else 0

        print(f"\nSitio: {nombre} ({lat:.4f}, {lon:.4f})")
        print(f"Año representativo (completo): {year_rep}")
        print(f"Año seleccionado (completo): {int(year)}")
        print(f"Escurrimiento base (año completo): {q_base_year:.2f} mm")
        print(f"Escurrimiento con obras (año completo): {q_obra_year:.2f} mm")
        print(f"Beneficio volumétrico (año completo): {diferencia_mm:.2f} mm   =   {diferencia_m3ha:.2f} m³/ha")
        print(f"Eventos con P > 0.2S base (año completo): {int(eventos_base_year)}")
        print(f"Eventos con P > 0.2S con obras (año completo): {int(eventos_obra_year)}")
        print(f"Coef. escurrimiento base: {coef_q_base:.2%}")
        print(f"Coef. escurrimiento con obras: {coef_q_obra:.2%}")

        # --- HIDROGRAMA SOLO DEL RANGO DE MESES Y VARIABLES SELECCIONADAS ---
        df_rep = df_anno[(df_anno['Month'] >= mes_ini) & (df_anno['Month'] <= mes_fin)]

        plt.figure(figsize=(15,6))
        if 'Precipitación' in variables:
            plt.plot(df_rep['Date'], df_rep['PCP'], color='navy', label='Precipitación (mm)', linewidth=1, alpha=0.6)
        if 'Escurrimiento base' in variables:
            plt.plot(df_rep['Date'], df_rep['Q_base'], color='red', label='Escurrimiento Base (mm)', linewidth=1)
        if 'Escurrimiento con obras' in variables:
            plt.plot(df_rep['Date'], df_rep['Q_obra'], color='green', label='Escurrimiento con obras (mm)', linewidth=1, linestyle='--')
        plt.title(f'Hidrograma año {int(year)}, meses {mes_ini}-{mes_fin} - {nombre}')
        plt.xlabel('Fecha')
        plt.ylabel('mm')
        plt.legend()
        plt.grid(alpha=0.3)
        plt.tight_layout()
        plt.show()

        # Comparación barra (del año completo)
        plt.figure(figsize=(10,5))
        plt.bar(['Base','Con obras','Beneficio'], [q_base_year, q_obra_year, diferencia_mm], color=['red','green','blue'], alpha=0.8)
        plt.ylabel('Escurrimiento anual (mm)')
        plt.title(f'Comparación escurrimiento y beneficio volumétrico\nAño {int(year)} - {nombre}')
        plt.grid(axis='y', alpha=0.2)
        plt.tight_layout()
        plt.show()

        # Exporta el año completo, ordenado por fecha
        global last_df_export
        export_cols = ['Date', 'PCP', 'Q_base', 'Q_obra']
        last_df_export = df_anno[export_cols].sort_values('Date').copy()

# --- Widgets interactivos ---
nombre_default = "Toroto"
lat_default = 19.418922222134
lon_default = -99.16033686822948
CN_base_default = 90
CN_obra_default = 80

year_rep, min_year, max_year, df_init = get_data_and_yearrep(lat_default, lon_default, CN_base_default)
nombre = Text(value=nombre_default, description="Nombre:")
lat = FloatText(value=lat_default, description="Latitud:")
lon = FloatText(value=lon_default, description="Longitud:")
CN_base = IntText(value=CN_base_default, description="CN base:")
CN_obra = IntText(value=CN_obra_default, description="CN obras:")
year = IntText(value=year_rep, description="Año")
mes_ini = IntSlider(value=1, min=1, max=12, description="Mes inicial:", continuous_update=False)
mes_fin = IntSlider(value=12, min=1, max=12, description="Mes final:", continuous_update=False)
prec_widget = Checkbox(value=True, description='Precipitación')
base_widget = Checkbox(value=True, description='Escurrimiento base')
obra_widget = Checkbox(value=True, description='Escurrimiento con obras')
btn_rep = Button(description="Año representativo", button_style='info')
btn_export = Button(description="Exportar datos", button_style='success')
output = Output()

last_df_export = None  # Global para exportación

def get_variables_selected():
    v = []
    if prec_widget.value:
        v.append('Precipitación')
    if base_widget.value:
        v.append('Escurrimiento base')
    if obra_widget.value:
        v.append('Escurrimiento con obras')
    return v

def actualizar_dashboard(change=None):
    if mes_ini.value > mes_fin.value:
        mes_ini.value, mes_fin.value = mes_fin.value, mes_ini.value
    run_dashboard(
        nombre.value, lat.value, lon.value, CN_base.value, CN_obra.value,
        year.value, mes_ini.value, mes_fin.value,
        get_variables_selected(), output
    )

for w in [nombre, lat, lon, CN_base, CN_obra, year, mes_ini, mes_fin, prec_widget, base_widget, obra_widget]:
    w.observe(actualizar_dashboard, names='value')

def set_year_rep(b):
    year_rep, _, _, _ = get_data_and_yearrep(lat.value, lon.value, CN_base.value)
    year.value = year_rep

btn_rep.on_click(set_year_rep)

def exportar_datos(b):
    global last_df_export
    if last_df_export is not None and len(last_df_export) > 0:
        fname = f"{nombre.value}_{year.value}.csv"
        last_df_export.to_csv(fname, index=False)
        print(f"Archivo exportado: {fname}")
    else:
        print("No hay datos para exportar. Cambia los parámetros y vuelve a intentar.")

btn_export.on_click(exportar_datos)

# Interfaz
display(VBox([
    HBox([nombre, lat, lon]),
    HBox([CN_base, CN_obra, year, btn_rep]),
    HBox([mes_ini, mes_fin]),
    HBox([prec_widget, base_widget, obra_widget]),
    btn_export,
    output
]))
actualizar_dashboard()

El sistema no puede encontrar la ruta especificada.


In [4]:
# @title
# Instala dependencias (si no las tienes) - ejecuta sólo una vez por sesión
!pip install pandas pydaymet datetime matplotlib ipywidgets > /dev/null

import pandas as pd
from pydaymet import get_bycoords
from datetime import date
import matplotlib.pyplot as plt
from ipywidgets import FloatText, IntText, Text, IntSlider, Button, HBox, VBox, Output, Checkbox
from IPython.display import display

def get_data_and_yearrep(lat, lon, CN_base):
    start = date(1980, 1, 1)
    end = date(date.today().year - 1, 12, 31)
    dates = (start.strftime("%Y-%m-%d"), end.strftime("%Y-%m-%d"))
    df = get_bycoords((lon, lat), dates, variables=['prcp'], time_scale='daily').reset_index()
    df = df.rename(columns={'time':'Date', 'prcp (mm/day)':'PCP'})[['Date', 'PCP']]
    df['PCP'] = df['PCP'].interpolate(method='linear', limit=1)
    df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d', errors='coerce')
    df = df.dropna(subset=['Date'])
    df['Year'] = df['Date'].dt.year
    df['Month'] = df['Date'].dt.month

    S_base = 25400 / CN_base - 254
    def runoff(P, S):
        return ((P - 0.2*S)**2)/(P + 0.8*S) if P > 0.2*S else 0
    df['Q_base'] = df['PCP'].apply(lambda P: runoff(P, S_base))
    annual_base = df.groupby('Year')['Q_base'].sum().reset_index(name='annual_Q_base')
    median_base = annual_base['annual_Q_base'].median()
    annual_base['diff_mediana'] = (annual_base['annual_Q_base'] - median_base).abs()
    min_diff = annual_base['diff_mediana'].min()
    candidatos = annual_base[annual_base['diff_mediana'] == min_diff]
    year_rep = candidatos['Year'].max()
    return int(year_rep), int(df['Year'].min()), int(df['Year'].max()), df

def run_dashboard(nombre, lat, lon, CN_base, CN_obra, year, mes_ini, mes_fin, variables, output):
    output.clear_output()
    with output:
        year_rep, min_year, max_year, df = get_data_and_yearrep(lat, lon, CN_base)
        if year < min_year or year > max_year:
            print(f"Año {year} no disponible. Elige entre {min_year} y {max_year}.")
            return

        S_base = 25400 / CN_base - 254
        S_obra = 25400 / CN_obra - 254
        def runoff(P, S):
            return ((P - 0.2*S)**2)/(P + 0.8*S) if P > 0.2*S else 0
        df['Q_base'] = df['PCP'].apply(lambda P: runoff(P, S_base))
        df['Q_obra'] = df['PCP'].apply(lambda P: runoff(P, S_obra))
        df['Evento_base'] = df['PCP'] > 0.2 * S_base
        df['Evento_obra'] = df['PCP'] > 0.2 * S_obra

        # --- Datos año completo para métricas y barras ---
        df_anno = df[df['Year'] == year]

        # Métricas del año completo
        q_base_year = df_anno['Q_base'].sum()
        q_obra_year = df_anno['Q_obra'].sum()
        diferencia_mm = q_base_year - q_obra_year
        diferencia_m3ha = diferencia_mm * 10
        eventos_base_year = df_anno['Evento_base'].sum()
        eventos_obra_year = df_anno['Evento_obra'].sum()
        pp_anual = df_anno['PCP'].sum()
        coef_q_base = q_base_year / pp_anual if pp_anual > 0 else 0
        coef_q_obra = q_obra_year / pp_anual if pp_anual > 0 else 0

        print(f"\nSitio: {nombre} ({lat:.4f}, {lon:.4f})")
        print(f"Año representativo (completo): {year_rep}")
        print(f"Año seleccionado (completo): {int(year)}")
        print(f"Escurrimiento base (año completo): {q_base_year:.2f} mm")
        print(f"Escurrimiento con obras (año completo): {q_obra_year:.2f} mm")
        print(f"Beneficio volumétrico (año completo): {diferencia_mm:.2f} mm   =   {diferencia_m3ha:.2f} m³/ha")
        print(f"Eventos con P > 0.2S base (año completo): {int(eventos_base_year)}")
        print(f"Eventos con P > 0.2S con obras (año completo): {int(eventos_obra_year)}")
        print(f"Coef. escurrimiento base: {coef_q_base:.2%}")
        print(f"Coef. escurrimiento con obras: {coef_q_obra:.2%}")

        # --- HIDROGRAMA SOLO DEL RANGO DE MESES Y VARIABLES SELECCIONADAS ---
        df_rep = df_anno[(df_anno['Month'] >= mes_ini) & (df_anno['Month'] <= mes_fin)]

        plt.figure(figsize=(15,6))
        if 'Precipitación' in variables:
            plt.plot(df_rep['Date'], df_rep['PCP'], color='navy', label='Precipitación (mm)', linewidth=1, alpha=0.6)
        if 'Escurrimiento base' in variables:
            plt.plot(df_rep['Date'], df_rep['Q_base'], color='red', label='Escurrimiento Base (mm)', linewidth=1)
        if 'Escurrimiento con obras' in variables:
            plt.plot(df_rep['Date'], df_rep['Q_obra'], color='green', label='Escurrimiento con obras (mm)', linewidth=1, linestyle='--')
        plt.title(f'Hidrograma año {int(year)}, meses {mes_ini}-{mes_fin} - {nombre}')
        plt.xlabel('Fecha')
        plt.ylabel('mm')
        plt.legend()
        plt.grid(alpha=0.3)
        plt.tight_layout()
        plt.show()

        # Comparación barra (del año completo)
        plt.figure(figsize=(10,5))
        plt.bar(['Base','Con obras','Beneficio'], [q_base_year, q_obra_year, diferencia_mm], color=['red','green','blue'], alpha=0.8)
        plt.ylabel('Escurrimiento anual (mm)')
        plt.title(f'Comparación escurrimiento y beneficio volumétrico\nAño {int(year)} - {nombre}')
        plt.grid(axis='y', alpha=0.2)
        plt.tight_layout()
        plt.show()

        # Exporta solo columnas seleccionadas y renombradas
        global last_df_export
        # Mapeo de variables y nombres
        col_map = {
            'Precipitación': ('PCP', 'Precipitación'),
            'Escurrimiento base': ('Q_base', 'Escurrimiento base'),
            'Escurrimiento con obras': ('Q_obra', 'Escurrimiento con obras')
        }
        export_cols = ['Date']
        export_names = ['Fecha']
        for var in variables:
            export_cols.append(col_map[var][0])
            export_names.append(col_map[var][1])
        # Si no se selecciona ninguna, exporta solo fecha
        last_df_export = df_anno[export_cols].sort_values('Date').copy()
        last_df_export.columns = export_names

# --- Widgets interactivos ---
nombre_default = "Toroto"
lat_default = 19.418922222134
lon_default = -99.16033686822948
CN_base_default = 90
CN_obra_default = 80

year_rep, min_year, max_year, df_init = get_data_and_yearrep(lat_default, lon_default, CN_base_default)
nombre = Text(value=nombre_default, description="Nombre:")
lat = FloatText(value=lat_default, description="Latitud:")
lon = FloatText(value=lon_default, description="Longitud:")
CN_base = IntText(value=CN_base_default, description="CN base:")
CN_obra = IntText(value=CN_obra_default, description="CN obras:")
year = IntText(value=year_rep, description="Año")
mes_ini = IntSlider(value=1, min=1, max=12, description="Mes inicial:", continuous_update=False)
mes_fin = IntSlider(value=12, min=1, max=12, description="Mes final:", continuous_update=False)
prec_widget = Checkbox(value=True, description='Precipitación')
base_widget = Checkbox(value=True, description='Escurrimiento base')
obra_widget = Checkbox(value=True, description='Escurrimiento con obras')
btn_rep = Button(description="Año representativo", button_style='info')
btn_export = Button(description="Exportar datos", button_style='success')
output = Output()

last_df_export = None  # Global para exportación

def get_variables_selected():
    v = []
    if prec_widget.value:
        v.append('Precipitación')
    if base_widget.value:
        v.append('Escurrimiento base')
    if obra_widget.value:
        v.append('Escurrimiento con obras')
    return v

def actualizar_dashboard(change=None):
    if mes_ini.value > mes_fin.value:
        mes_ini.value, mes_fin.value = mes_fin.value, mes_ini.value
    run_dashboard(
        nombre.value, lat.value, lon.value, CN_base.value, CN_obra.value,
        year.value, mes_ini.value, mes_fin.value,
        get_variables_selected(), output
    )

for w in [nombre, lat, lon, CN_base, CN_obra, year, mes_ini, mes_fin, prec_widget, base_widget, obra_widget]:
    w.observe(actualizar_dashboard, names='value')

def set_year_rep(b):
    year_rep, _, _, _ = get_data_and_yearrep(lat.value, lon.value, CN_base.value)
    year.value = year_rep

btn_rep.on_click(set_year_rep)

def exportar_datos(b):
    global last_df_export
    if last_df_export is not None and len(last_df_export) > 0:
        fname = f"{nombre.value}_{year.value}.csv"
        last_df_export.to_csv(fname, index=False)
        print(f"Archivo exportado: {fname}")
    else:
        print("No hay datos para exportar. Cambia los parámetros y vuelve a intentar.")

btn_export.on_click(exportar_datos)

# Interfaz
display(VBox([
    HBox([nombre, lat, lon]),
    HBox([CN_base, CN_obra, year, btn_rep]),
    HBox([mes_ini, mes_fin]),
    HBox([prec_widget, base_widget, obra_widget]),
    btn_export,
    output
]))
actualizar_dashboard()

El sistema no puede encontrar la ruta especificada.


In [13]:
# @title
# Instala dependencias (si no las tienes) - ejecuta sólo una vez por sesión
!pip install pandas pydaymet datetime matplotlib ipywidgets openpyxl > /dev/null

import pandas as pd
from pydaymet import get_bycoords
from datetime import date
import matplotlib.pyplot as plt
from ipywidgets import FloatText, IntText, Text, IntSlider, Button, HBox, VBox, Output, Checkbox
from IPython.display import display

def get_data_and_yearrep(lat, lon, CN_base):
    start = date(1980, 1, 1)
    end = date(date.today().year - 1, 12, 31)
    dates = (start.strftime("%Y-%m-%d"), end.strftime("%Y-%m-%d"))
    df = get_bycoords((lon, lat), dates, variables=['prcp'], time_scale='daily').reset_index()
    df = df.rename(columns={'time':'Date', 'prcp (mm/day)':'PCP'})[['Date', 'PCP']]
    df['PCP'] = df['PCP'].interpolate(method='linear', limit=1)
    df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d', errors='coerce')
    df = df.dropna(subset=['Date'])
    df['Year'] = df['Date'].dt.year
    df['Month'] = df['Date'].dt.month

    S_base = 25400 / CN_base - 254
    def runoff(P, S):
        return ((P - 0.2*S)**2)/(P + 0.8*S) if P > 0.2*S else 0
    df['Q_base'] = df['PCP'].apply(lambda P: runoff(P, S_base))
    annual_base = df.groupby('Year')['Q_base'].sum().reset_index(name='annual_Q_base')
    median_base = annual_base['annual_Q_base'].median()
    annual_base['diff_mediana'] = (annual_base['annual_Q_base'] - median_base).abs()
    min_diff = annual_base['diff_mediana'].min()
    candidatos = annual_base[annual_base['diff_mediana'] == min_diff]
    year_rep = candidatos['Year'].max()
    return int(year_rep), int(df['Year'].min()), int(df['Year'].max()), df

def run_dashboard(nombre, lat, lon, CN_base, CN_obra, year, mes_ini, mes_fin, variables, output):
    output.clear_output()
    with output:
        year_rep, min_year, max_year, df = get_data_and_yearrep(lat, lon, CN_base)
        if year < min_year or year > max_year:
            print(f"Año {year} no disponible. Elige entre {min_year} y {max_year}.")
            return

        S_base = 25400 / CN_base - 254
        S_obra = 25400 / CN_obra - 254
        def runoff(P, S):
            return ((P - 0.2*S)**2)/(P + 0.8*S) if P > 0.2*S else 0
        df['Q_base'] = df['PCP'].apply(lambda P: runoff(P, S_base))
        df['Q_obra'] = df['PCP'].apply(lambda P: runoff(P, S_obra))
        df['Evento_base'] = df['PCP'] > 0.2 * S_base
        df['Evento_obra'] = df['PCP'] > 0.2 * S_obra

        # --- Datos año completo para métricas y barras ---
        df_anno = df[df['Year'] == year]

        # Métricas del año completo
        q_base_year = df_anno['Q_base'].sum()
        q_obra_year = df_anno['Q_obra'].sum()
        diferencia_mm = q_base_year - q_obra_year
        diferencia_m3ha = diferencia_mm * 10
        eventos_base_year = df_anno['Evento_base'].sum()
        eventos_obra_year = df_anno['Evento_obra'].sum()
        pp_anual = df_anno['PCP'].sum()
        coef_q_base = q_base_year / pp_anual if pp_anual > 0 else 0
        coef_q_obra = q_obra_year / pp_anual if pp_anual > 0 else 0

        print(f"\nSitio: {nombre} ({lat:.4f}, {lon:.4f})")
        print(f"Año representativo (completo): {year_rep}")
        print(f"Año seleccionado (completo): {int(year)}")
        print(f"Escurrimiento base (año completo): {q_base_year:.2f} mm")
        print(f"Escurrimiento con obras (año completo): {q_obra_year:.2f} mm")
        print(f"Beneficio volumétrico (año completo): {diferencia_mm:.2f} mm   =   {diferencia_m3ha:.2f} m³/ha")
        print(f"Eventos con P > 0.2S base (año completo): {int(eventos_base_year)}")
        print(f"Eventos con P > 0.2S con obras (año completo): {int(eventos_obra_year)}")
        print(f"Coef. escurrimiento base: {coef_q_base:.2%}")
        print(f"Coef. escurrimiento con obras: {coef_q_obra:.2%}")

        # --- HIDROGRAMA SOLO DEL RANGO DE MESES Y VARIABLES SELECCIONADAS ---
        df_rep = df_anno[(df_anno['Month'] >= mes_ini) & (df_anno['Month'] <= mes_fin)]

        plt.figure(figsize=(15,6))
        if 'Precipitación' in variables:
            plt.plot(df_rep['Date'], df_rep['PCP'], color='#0055B8', label='Precipitación (mm)', linewidth=1, alpha=0.6)
        if 'Escurrimiento base' in variables:
            plt.plot(df_rep['Date'], df_rep['Q_base'], color='#FF6469', label='Escurrimiento Base (mm)', linewidth=1)
        if 'Escurrimiento con obras' in variables:
            plt.plot(df_rep['Date'], df_rep['Q_obra'], color='#32CC68', label='Escurrimiento con obras (mm)', linewidth=1)
        plt.title(f'Hidrograma año representativo: {int(year)}, meses {mes_ini}-{mes_fin} - {nombre}')
        plt.xlabel('Fecha')
        plt.ylabel('mm')
        plt.legend()
        plt.grid(alpha=0.3)
        plt.tight_layout()
        plt.show()

        # Comparación barra (del año completo)
        plt.figure(figsize=(10,5))
        plt.bar(['Base','Con obras','Beneficio'], [q_base_year, q_obra_year, diferencia_mm], color=['#FF6469','#80CE9F','#0055B8'], alpha=0.8)
        plt.ylabel('Escurrimiento anual (mm)')
        plt.title(f'Comparación escurrimiento y beneficio volumétrico\nAño representativo: {int(year)} - {nombre}')
        plt.grid(axis='y', alpha=0.2)
        plt.tight_layout()
        plt.show()

        # Exporta SIEMPRE las tres variables y fecha, en Excel con nombres bien escritos
        global last_df_export
        export_cols = ['Date', 'PCP', 'Q_base', 'Q_obra']
        export_names = ['Fecha', 'Precipitación', 'Escurrimiento base', 'Escurrimiento con obras']
        last_df_export = df_anno[export_cols].sort_values('Date').copy()
        last_df_export.columns = export_names

# --- Widgets interactivos ---
nombre_default = "Toroto"
lat_default = 19.418922222134
lon_default = -99.16033686822948
CN_base_default = 90
CN_obra_default = 80

year_rep, min_year, max_year, df_init = get_data_and_yearrep(lat_default, lon_default, CN_base_default)
nombre = Text(value=nombre_default, description="Nombre:")
lat = FloatText(value=lat_default, description="Latitud:")
lon = FloatText(value=lon_default, description="Longitud:")
CN_base = IntText(value=CN_base_default, description="CN base:")
CN_obra = IntText(value=CN_obra_default, description="CN obras:")
year = IntText(value=year_rep, description="Año")
mes_ini = IntSlider(value=1, min=1, max=12, description="Mes inicial:", continuous_update=False)
mes_fin = IntSlider(value=12, min=1, max=12, description="Mes final:", continuous_update=False)
prec_widget = Checkbox(value=True, description='Precipitación')
base_widget = Checkbox(value=True, description='Escurrimiento base')
obra_widget = Checkbox(value=True, description='Escurrimiento con obras')
btn_rep = Button(description="Año representativo", button_style='info')
btn_export = Button(description="Exportar datos (.xlsx)", button_style='success')
output = Output()

last_df_export = None  # Global para exportación

def get_variables_selected():
    v = []
    if prec_widget.value:
        v.append('Precipitación')
    if base_widget.value:
        v.append('Escurrimiento base')
    if obra_widget.value:
        v.append('Escurrimiento con obras')
    return v

def actualizar_dashboard(change=None):
    if mes_ini.value > mes_fin.value:
        mes_ini.value, mes_fin.value = mes_fin.value, mes_ini.value
    run_dashboard(
        nombre.value, lat.value, lon.value, CN_base.value, CN_obra.value,
        year.value, mes_ini.value, mes_fin.value,
        get_variables_selected(), output
    )

for w in [nombre, lat, lon, CN_base, CN_obra, year, mes_ini, mes_fin, prec_widget, base_widget, obra_widget]:
    w.observe(actualizar_dashboard, names='value')

def set_year_rep(b):
    year_rep, _, _, _ = get_data_and_yearrep(lat.value, lon.value, CN_base.value)
    year.value = year_rep

btn_rep.on_click(set_year_rep)

def exportar_datos(b):
    global last_df_export
    if last_df_export is not None and len(last_df_export) > 0:
        fname = f"{nombre.value}_{year.value}.xlsx"
        last_df_export.to_excel(fname, index=False)
        print(f"Archivo exportado: {fname}")
    else:
        print("No hay datos para exportar. Cambia los parámetros y vuelve a intentar.")

btn_export.on_click(exportar_datos)

# Interfaz
display(VBox([
    HBox([nombre, lat, lon]),
    HBox([CN_base, CN_obra, year, btn_rep]),
    HBox([mes_ini, mes_fin]),
    HBox([prec_widget, base_widget, obra_widget]),
    btn_export,
    output
]))
actualizar_dashboard()

El sistema no puede encontrar la ruta especificada.


# Bueno

In [14]:
# @title
# Instalar dependencias (ejecutar solo una vez por sesión)
!pip install pandas pydaymet datetime matplotlib ipywidgets openpyxl > /dev/null

import pandas as pd
from pydaymet import get_bycoords
from datetime import date
import matplotlib.pyplot as plt
from ipywidgets import FloatText, IntText, Text, IntSlider, Button, HBox, VBox, Output, Checkbox
from IPython.display import display

def get_data_and_yearrep(lat, lon, CN_base):
    # Descargar datos diarios de Daymet y calcular año representativo
    start = date(1980, 1, 1)
    end = date(date.today().year - 1, 12, 31)
    dates = (start.strftime("%Y-%m-%d"), end.strftime("%Y-%m-%d"))
    df = get_bycoords((lon, lat), dates, variables=['prcp'], time_scale='daily').reset_index()
    df = df.rename(columns={'time':'Date', 'prcp (mm/day)':'PCP'})[['Date', 'PCP']]
    df['PCP'] = df['PCP'].interpolate(method='linear', limit=1)
    df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d', errors='coerce')
    df = df.dropna(subset=['Date'])
    df['Year'] = df['Date'].dt.year
    df['Month'] = df['Date'].dt.month

    # Calcular escurrimiento base usando el método SCS-CN
    S_base = 25400 / CN_base - 254
    def runoff(P, S):
        return ((P - 0.2*S)**2)/(P + 0.8*S) if P > 0.2*S else 0
    df['Q_base'] = df['PCP'].apply(lambda P: runoff(P, S_base))

    # Encontrar año representativo según mediana anual de escurrimiento base
    annual_base = df.groupby('Year')['Q_base'].sum().reset_index(name='annual_Q_base')
    median_base = annual_base['annual_Q_base'].median()
    annual_base['diff_mediana'] = (annual_base['annual_Q_base'] - median_base).abs()
    min_diff = annual_base['diff_mediana'].min()
    candidatos = annual_base[annual_base['diff_mediana'] == min_diff]
    year_rep = candidatos['Year'].max()
    return int(year_rep), int(df['Year'].min()), int(df['Year'].max()), df

def run_dashboard(nombre, lat, lon, CN_base, CN_obra, year, mes_ini, mes_fin, variables, output):
    # Limpiar salida previa
    output.clear_output()
    with output:
        # Obtener datos y año representativo
        year_rep, min_year, max_year, df = get_data_and_yearrep(lat, lon, CN_base)
        if year < min_year or year > max_year:
            print(f"Año {year} no disponible. Elegir entre {min_year} y {max_year}.")
            return

        # Calcular escurrimiento base y con obras
        S_base = 25400 / CN_base - 254
        S_obra = 25400 / CN_obra - 254
        def runoff(P, S):
            return ((P - 0.2*S)**2)/(P + 0.8*S) if P > 0.2*S else 0
        df['Q_base'] = df['PCP'].apply(lambda P: runoff(P, S_base))
        df['Q_obra'] = df['PCP'].apply(lambda P: runoff(P, S_obra))
        df['Evento_base'] = df['PCP'] > 0.2 * S_base
        df['Evento_obra'] = df['PCP'] > 0.2 * S_obra

        # Filtrar año completo para métricas y barras
        df_anno = df[df['Year'] == year]

        # Calcular métricas anuales
        q_base_year = df_anno['Q_base'].sum()
        q_obra_year = df_anno['Q_obra'].sum()
        diferencia_mm = q_base_year - q_obra_year
        diferencia_m3ha = diferencia_mm * 10
        eventos_base_year = df_anno['Evento_base'].sum()
        eventos_obra_year = df_anno['Evento_obra'].sum()
        pp_anual = df_anno['PCP'].sum()
        coef_q_base = q_base_year / pp_anual if pp_anual > 0 else 0
        coef_q_obra = q_obra_year / pp_anual if pp_anual > 0 else 0

        # Mostrar resumen de métricas anuales
        print(f"\nSitio: {nombre} ({lat:.4f}, {lon:.4f})")
        print(f"Año representativo (completo): {year_rep}")
        print(f"Año seleccionado (completo): {int(year)}")
        print(f"Escurrimiento base (año completo): {q_base_year:.2f} mm")
        print(f"Escurrimiento con obras (año completo): {q_obra_year:.2f} mm")
        print(f"Beneficio volumétrico (año completo): {diferencia_mm:.2f} mm   =   {diferencia_m3ha:.2f} m³/ha")
        print(f"Eventos con P > 0.2S base (año completo): {int(eventos_base_year)}")
        print(f"Eventos con P > 0.2S con obras (año completo): {int(eventos_obra_year)}")
        print(f"Coef. escurrimiento base: {coef_q_base:.2%}")
        print(f"Coef. escurrimiento con obras: {coef_q_obra:.2%}")

        # Filtrar rango de meses para hidrograma mostrado
        df_rep = df_anno[(df_anno['Month'] >= mes_ini) & (df_anno['Month'] <= mes_fin)]

        # Graficar hidrograma del rango de meses seleccionado
        plt.figure(figsize=(15,6))
        if 'Precipitación' in variables:
            plt.plot(df_rep['Date'], df_rep['PCP'], color='#0055B8', label='Precipitación (mm)', linewidth=1, alpha=0.6)
        if 'Escurrimiento base' in variables:
            plt.plot(df_rep['Date'], df_rep['Q_base'], color='#FF6469', label='Escurrimiento Base (mm)', linewidth=1)
        if 'Escurrimiento con obras' in variables:
            plt.plot(df_rep['Date'], df_rep['Q_obra'], color='#32CC68', label='Escurrimiento con obras (mm)', linewidth=1)
        plt.title(f'Hidrograma año representativo: {int(year)}, meses {mes_ini}-{mes_fin} - {nombre}')
        plt.xlabel('Fecha')
        plt.ylabel('mm')
        plt.legend()
        plt.grid(alpha=0.3)
        plt.tight_layout()
        plt.show()

        # Graficar barras comparativas anuales
        plt.figure(figsize=(10,5))
        plt.bar(['Base','Con obras','Beneficio'], [q_base_year, q_obra_year, diferencia_mm], color=['#FF6469','#80CE9F','#0055B8'], alpha=0.8)
        plt.ylabel('Escurrimiento anual (mm)')
        plt.title(f'Comparación escurrimiento y beneficio volumétrico\nAño representativo: {int(year)} - {nombre}')
        plt.grid(axis='y', alpha=0.2)
        plt.tight_layout()
        plt.show()

        # Preparar datos exportables: siempre exportar Fecha, Precipitación, Escurrimiento base y con obras
        global last_df_export
        export_cols = ['Date', 'PCP', 'Q_base', 'Q_obra']
        export_names = ['Fecha', 'Precipitación', 'Escurrimiento base', 'Escurrimiento con obras']
        last_df_export = df_anno[export_cols].sort_values('Date').copy()
        last_df_export.columns = export_names

# --- Definir widgets interactivos ---
nombre_default = "Toroto"
lat_default = 19.418922222134
lon_default = -99.16033686822948
CN_base_default = 90
CN_obra_default = 80

# Obtener año representativo y rango de años
year_rep, min_year, max_year, df_init = get_data_and_yearrep(lat_default, lon_default, CN_base_default)
nombre = Text(value=nombre_default, description="Nombre:")
lat = FloatText(value=lat_default, description="Latitud:")
lon = FloatText(value=lon_default, description="Longitud:")
CN_base = IntText(value=CN_base_default, description="CN base:")
CN_obra = IntText(value=CN_obra_default, description="CN obras:")
year = IntText(value=year_rep, description="Año")
mes_ini = IntSlider(value=1, min=1, max=12, description="Mes inicial:", continuous_update=False)
mes_fin = IntSlider(value=12, min=1, max=12, description="Mes final:", continuous_update=False)
prec_widget = Checkbox(value=True, description='Precipitación')
base_widget = Checkbox(value=True, description='Escurrimiento base')
obra_widget = Checkbox(value=True, description='Escurrimiento con obras')
btn_rep = Button(description="Año representativo", button_style='info')
btn_export = Button(description="Exportar datos (.xlsx)", button_style='success')
output = Output()

last_df_export = None  # Variable global para exportación

def get_variables_selected():
    # Obtener variables seleccionadas para mostrar en la gráfica
    v = []
    if prec_widget.value:
        v.append('Precipitación')
    if base_widget.value:
        v.append('Escurrimiento base')
    if obra_widget.value:
        v.append('Escurrimiento con obras')
    return v

def actualizar_dashboard(change=None):
    # Actualizar dashboard cuando cambia algún parámetro
    if mes_ini.value > mes_fin.value:
        mes_ini.value, mes_fin.value = mes_fin.value, mes_ini.value
    run_dashboard(
        nombre.value, lat.value, lon.value, CN_base.value, CN_obra.value,
        year.value, mes_ini.value, mes_fin.value,
        get_variables_selected(), output
    )

for w in [nombre, lat, lon, CN_base, CN_obra, year, mes_ini, mes_fin, prec_widget, base_widget, obra_widget]:
    w.observe(actualizar_dashboard, names='value')

def set_year_rep(b):
    # Asignar año representativo como año seleccionado
    year_rep, _, _, _ = get_data_and_yearrep(lat.value, lon.value, CN_base.value)
    year.value = year_rep

btn_rep.on_click(set_year_rep)

def exportar_datos(b):
    # Exportar datos anuales a archivo Excel
    global last_df_export
    if last_df_export is not None and len(last_df_export) > 0:
        fname = f"{nombre.value}_{year.value}.xlsx"
        last_df_export.to_excel(fname, index=False)
        print(f"Archivo exportado: {fname}")
    else:
        print("No hay datos para exportar. Cambiar parámetros y volver a intentar.")

btn_export.on_click(exportar_datos)

# Mostrar interfaz gráfica interactiva
display(VBox([
    HBox([nombre, lat, lon]),
    HBox([CN_base, CN_obra, year, btn_rep]),
    HBox([mes_ini, mes_fin]),
    HBox([prec_widget, base_widget, obra_widget]),
    btn_export,
    output
]))
actualizar_dashboard()

El sistema no puede encontrar la ruta especificada.


In [15]:
# @title
# Instalar dependencias (ejecutar solo una vez por sesión)
!pip install pandas pydaymet datetime matplotlib ipywidgets openpyxl > /dev/null
!wget -q https://github.com/google/fonts/raw/main/ofl/ruda/Ruda-Regular.ttf
!wget -q https://github.com/google/fonts/raw/main/ofl/dmsans/DMSans-Regular.ttf

import pandas as pd
from pydaymet import get_bycoords
from datetime import date
import matplotlib.pyplot as plt
from ipywidgets import FloatText, IntText, Text, IntSlider, Button, HBox, VBox, Output, Checkbox
from IPython.display import display
from matplotlib.font_manager import FontProperties

# Definir tipografías para encabezados y cuerpo de texto
fp_ruda = FontProperties(fname='Ruda-Regular.ttf')
fp_dm = FontProperties(fname='DMSans-Regular.ttf')

def get_data_and_yearrep(lat, lon, CN_base):
    # Descargar datos diarios de Daymet y calcular año representativo
    start = date(1980, 1, 1)
    end = date(date.today().year - 1, 12, 31)
    dates = (start.strftime("%Y-%m-%d"), end.strftime("%Y-%m-%d"))
    df = get_bycoords((lon, lat), dates, variables=['prcp'], time_scale='daily').reset_index()
    df = df.rename(columns={'time':'Date', 'prcp (mm/day)':'PCP'})[['Date', 'PCP']]
    df['PCP'] = df['PCP'].interpolate(method='linear', limit=1)
    df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d', errors='coerce')
    df = df.dropna(subset=['Date'])
    df['Year'] = df['Date'].dt.year
    df['Month'] = df['Date'].dt.month

    # Calcular escurrimiento base usando el método SCS-CN
    S_base = 25400 / CN_base - 254
    def runoff(P, S):
        return ((P - 0.2*S)**2)/(P + 0.8*S) if P > 0.2*S else 0
    df['Q_base'] = df['PCP'].apply(lambda P: runoff(P, S_base))

    # Encontrar año representativo según mediana anual de escurrimiento base
    annual_base = df.groupby('Year')['Q_base'].sum().reset_index(name='annual_Q_base')
    median_base = annual_base['annual_Q_base'].median()
    annual_base['diff_mediana'] = (annual_base['annual_Q_base'] - median_base).abs()
    min_diff = annual_base['diff_mediana'].min()
    candidatos = annual_base[annual_base['diff_mediana'] == min_diff]
    year_rep = candidatos['Year'].max()
    return int(year_rep), int(df['Year'].min()), int(df['Year'].max()), df

def run_dashboard(nombre, lat, lon, CN_base, CN_obra, year, mes_ini, mes_fin, variables, output):
    # Limpiar salida previa
    output.clear_output()
    with output:
        # Obtener datos y año representativo
        year_rep, min_year, max_year, df = get_data_and_yearrep(lat, lon, CN_base)
        if year < min_year or year > max_year:
            print(f"Año {year} no disponible. Elegir entre {min_year} y {max_year}.")
            return

        # Calcular escurrimiento base y con obras
        S_base = 25400 / CN_base - 254
        S_obra = 25400 / CN_obra - 254
        def runoff(P, S):
            return ((P - 0.2*S)**2)/(P + 0.8*S) if P > 0.2*S else 0
        df['Q_base'] = df['PCP'].apply(lambda P: runoff(P, S_base))
        df['Q_obra'] = df['PCP'].apply(lambda P: runoff(P, S_obra))
        df['Evento_base'] = df['PCP'] > 0.2 * S_base
        df['Evento_obra'] = df['PCP'] > 0.2 * S_obra

        # Filtrar año completo para métricas y barras
        df_anno = df[df['Year'] == year]

        # Calcular métricas anuales
        q_base_year = df_anno['Q_base'].sum()
        q_obra_year = df_anno['Q_obra'].sum()
        diferencia_mm = q_base_year - q_obra_year
        diferencia_m3ha = diferencia_mm * 10
        eventos_base_year = df_anno['Evento_base'].sum()
        eventos_obra_year = df_anno['Evento_obra'].sum()
        pp_anual = df_anno['PCP'].sum()
        coef_q_base = q_base_year / pp_anual if pp_anual > 0 else 0
        coef_q_obra = q_obra_year / pp_anual if pp_anual > 0 else 0

        # Mostrar resumen de métricas anuales
        print(f"\nSitio: {nombre} ({lat:.4f}, {lon:.4f})")
        print(f"Año representativo (completo): {year_rep}")
        print(f"Año seleccionado (completo): {int(year)}")
        print(f"Escurrimiento base (año completo): {q_base_year:.2f} mm")
        print(f"Escurrimiento con obras (año completo): {q_obra_year:.2f} mm")
        print(f"Beneficio volumétrico (año completo): {diferencia_mm:.2f} mm   =   {diferencia_m3ha:.2f} m³/ha")
        print(f"Eventos con P > 0.2S base (año completo): {int(eventos_base_year)}")
        print(f"Eventos con P > 0.2S con obras (año completo): {int(eventos_obra_year)}")
        print(f"Coef. escurrimiento base: {coef_q_base:.2%}")
        print(f"Coef. escurrimiento con obras: {coef_q_obra:.2%}")

        # Filtrar rango de meses para hidrograma mostrado
        df_rep = df_anno[(df_anno['Month'] >= mes_ini) & (df_anno['Month'] <= mes_fin)]

        # Graficar hidrograma del rango de meses seleccionado
        plt.figure(figsize=(15,6))
        if 'Precipitación' in variables:
            plt.plot(df_rep['Date'], df_rep['PCP'], color='#0055B8', label='Precipitación (mm)', linewidth=1, alpha=0.6)
        if 'Escurrimiento base' in variables:
            plt.plot(df_rep['Date'], df_rep['Q_base'], color='#FF6469', label='Escurrimiento Base (mm)', linewidth=1)
        if 'Escurrimiento con obras' in variables:
            plt.plot(df_rep['Date'], df_rep['Q_obra'], color='#32CC68', label='Escurrimiento con obras (mm)', linewidth=1)
        plt.title(f'Hidrograma año representativo: {int(year)}, meses {mes_ini}-{mes_fin} - {nombre}', fontproperties=fp_ruda, fontsize=18)
        plt.xlabel('Fecha', fontproperties=fp_dm, fontsize=13)
        plt.ylabel('mm', fontproperties=fp_dm, fontsize=13)
        plt.legend(prop=fp_dm)
        plt.xticks(fontproperties=fp_dm)
        plt.yticks(fontproperties=fp_dm)
        plt.grid(alpha=0.3)
        plt.tight_layout()
        plt.show()

        # Graficar barras comparativas anuales
        plt.figure(figsize=(10,5))
        plt.bar(['Base','Con obras','Beneficio'], [q_base_year, q_obra_year, diferencia_mm], color=['#FF6469','#80CE9F','#0055B8'], alpha=0.8)
        plt.ylabel('Escurrimiento anual (mm)', fontproperties=fp_dm, fontsize=13)
        plt.title(f'Comparación escurrimiento y beneficio volumétrico\nAño representativo: {int(year)} - {nombre}', fontproperties=fp_ruda, fontsize=16)
        plt.xticks(fontproperties=fp_dm)
        plt.yticks(fontproperties=fp_dm)
        plt.grid(axis='y', alpha=0.2)
        plt.tight_layout()
        plt.show()

        # Preparar datos exportables: siempre exportar Fecha, Precipitación, Escurrimiento base y con obras
        global last_df_export
        export_cols = ['Date', 'PCP', 'Q_base', 'Q_obra']
        export_names = ['Fecha', 'Precipitación', 'Escurrimiento base', 'Escurrimiento con obras']
        last_df_export = df_anno[export_cols].sort_values('Date').copy()
        last_df_export.columns = export_names

# Definir widgets interactivos
nombre_default = "Toroto"
lat_default = 19.418922222134
lon_default = -99.16033686822948
CN_base_default = 90
CN_obra_default = 80

# Obtener año representativo y rango de años
year_rep, min_year, max_year, df_init = get_data_and_yearrep(lat_default, lon_default, CN_base_default)
nombre = Text(value=nombre_default, description="Nombre:")
lat = FloatText(value=lat_default, description="Latitud:")
lon = FloatText(value=lon_default, description="Longitud:")
CN_base = IntText(value=CN_base_default, description="CN base:")
CN_obra = IntText(value=CN_obra_default, description="CN obras:")
year = IntText(value=year_rep, description="Año")
mes_ini = IntSlider(value=1, min=1, max=12, description="Mes inicial:", continuous_update=False)
mes_fin = IntSlider(value=12, min=1, max=12, description="Mes final:", continuous_update=False)
prec_widget = Checkbox(value=True, description='Precipitación')
base_widget = Checkbox(value=True, description='Escurrimiento base')
obra_widget = Checkbox(value=True, description='Escurrimiento con obras')
btn_rep = Button(description="Año representativo", button_style='info')
btn_export = Button(description="Exportar datos (.xlsx)", button_style='success')
output = Output()

last_df_export = None  # Variable global para exportación

def get_variables_selected():
    # Obtener variables seleccionadas para mostrar en la gráfica
    v = []
    if prec_widget.value:
        v.append('Precipitación')
    if base_widget.value:
        v.append('Escurrimiento base')
    if obra_widget.value:
        v.append('Escurrimiento con obras')
    return v

def actualizar_dashboard(change=None):
    # Actualizar dashboard cuando cambia algún parámetro
    if mes_ini.value > mes_fin.value:
        mes_ini.value, mes_fin.value = mes_fin.value, mes_ini.value
    run_dashboard(
        nombre.value, lat.value, lon.value, CN_base.value, CN_obra.value,
        year.value, mes_ini.value, mes_fin.value,
        get_variables_selected(), output
    )

for w in [nombre, lat, lon, CN_base, CN_obra, year, mes_ini, mes_fin, prec_widget, base_widget, obra_widget]:
    w.observe(actualizar_dashboard, names='value')

def set_year_rep(b):
    # Asignar año representativo como año seleccionado
    year_rep, _, _, _ = get_data_and_yearrep(lat.value, lon.value, CN_base.value)
    year.value = year_rep

btn_rep.on_click(set_year_rep)

def exportar_datos(b):
    # Exportar datos anuales a archivo Excel
    global last_df_export
    if last_df_export is not None and len(last_df_export) > 0:
        fname = f"{nombre.value}_{year.value}.xlsx"
        last_df_export.to_excel(fname, index=False)
        print(f"Archivo exportado: {fname}")
    else:
        print("No hay datos para exportar. Cambiar parámetros y volver a intentar.")

btn_export.on_click(exportar_datos)

# Mostrar interfaz gráfica interactiva
display(VBox([
    HBox([nombre, lat, lon]),
    HBox([CN_base, CN_obra, year, btn_rep]),
    HBox([mes_ini, mes_fin]),
    HBox([prec_widget, base_widget, obra_widget]),
    btn_export,
    output
]))
actualizar_dashboard()

El sistema no puede encontrar la ruta especificada.
"wget" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.
"wget" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


Error in callback <function _draw_all_if_interactive at 0x00000237D61FA3E0> (for post_execute), with arguments args (),kwargs {}:


FileNotFoundError: [Errno 2] No such file or directory: 'G:\\Mi unidad\\SeTE\\Optimizacion\\Beneficio volumétrico\\Prospeccion\\DMSans-Regular.ttf'

FileNotFoundError: [Errno 2] No such file or directory: 'G:\\Mi unidad\\SeTE\\Optimizacion\\Beneficio volumétrico\\Prospeccion\\DMSans-Regular.ttf'

<Figure size 1500x600 with 1 Axes>